### RAG sentencias judiciales




In [ ]:
# FASE 0: Webscrapping
import os
import requests
import unicodedata
import re
import time
import json

def slugify(value, allow_unicode=False):
    """
    Convierte un string en un "slug" seguro para nombres de archivo.
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

def setup_directory(folder_path):
    """
    Asegura que el directorio de destino exista y se mueva a él.
    """
    if not os.path.exists(folder_path):
        try:
            os.makedirs(folder_path)
            print(f"Carpeta creada en: {folder_path}")
        except OSError as e:
            print(f"Error al crear la carpeta {folder_path}: {e}")
            return False
    
    try:
        os.chdir(folder_path)
        print(f"Directorio de trabajo actual: {os.getcwd()}")
        return True
    except OSError as e:
        print(f"Error al cambiar al directorio {folder_path}: {e}")
        return False

def get_total_pages(api_url):
    """
    Obtiene el número total de páginas de la API.
    """
    try:
        print("Obteniendo número total de páginas...")
        page_response = requests.get(api_url)
        page_response.raise_for_status() # Verificar si hay errores
        last_page = page_response.json()['data']['tocas']['last_page']
        print(f"Total de páginas a procesar: {last_page}")
        return last_page
    except requests.RequestException as e:
        print(f"Error al obtener la paginación: {e}")
        return None
    except KeyError:
        print("Error: No se pudo encontrar 'last_page' en la respuesta de la API.")
        return None
    except json.JSONDecodeError:
        print("Error: La respuesta de la API no es un JSON válido.")
        return None

def download_sentencias(last_page, api_base_url):
    """
    Descarga todas las sentencias página por página.
    """
    for page_number in range(1, last_page + 1):
        print(f"\n--- Procesando página {page_number} de {last_page} ---")
        
        current_page_url = f"{api_base_url}/tocas?page={page_number}"
        
        try:
            page = requests.get(current_page_url)
            
            # Manejo de rate limiting (Error 429)
            if page.status_code == 429:
                retry_after = int(page.headers.get('retry-after', 30))
                print(f"Rate limit alcanzado. Esperando {retry_after} segundos...")
                time.sleep(retry_after)
                page = requests.get(current_page_url) # Reintentar
            
            page.raise_for_status()
            tocas_data = page.json()['data']['tocas']['data']
            
        except requests.RequestException as e:
            print(f"Error al obtener la página {page_number}: {e}. Saltando esta página.")
            continue
        except (KeyError, json.JSONDecodeError):
            print(f"Error de formato en la respuesta de la página {page_number}. Saltando.")
            continue

        # Iterar sobre cada sentencia (toca) en la página
        for toca in tocas_data:
            try:
                # --- 1. Validar y construir el nombre del archivo ---
                toca_id = toca.get('id')
                materia_data = toca.get('materia_data')
                delito_data = toca.get('delito_data')
                fecha = toca.get('fecha_publicacion') # Ej: "2024-01-08-103634"

                # Validar metadatos esenciales ANTES de acceder a sub-claves
                if not all([toca_id, materia_data, delito_data, fecha]):
                    print(f"Error: Faltan metadatos esenciales (ID, materia, delito o fecha) para Toca ID {toca_id}. Se omite.")
                    continue
                
                # Acceso seguro a sub-claves
                materia = materia_data.get('nombre')
                delito = delito_data.get('nombre')

                # Validar que las sub-claves también existan
                if not all([materia, delito]):
                    print(f"Error: Faltan 'nombre' en materia/delito para Toca ID {toca_id}. Se omite.")
                    continue
                
                # Construir la lista de partes del nombre
                partes_nombre = [fecha, materia, delito]
                
                # Añadir partes opcionales de forma segura
                accion_especifica = toca.get("accion_especifica_data")
                if accion_especifica and accion_especifica.get('nombre'):
                    partes_nombre.append(accion_especifica['nombre'])
                
                accion_simultanea = toca.get("accion_simultanea_data")
                if accion_simultanea and accion_simultanea.get('nombre'):
                    partes_nombre.append(accion_simultanea['nombre'])
                
                # Crear el nombre de archivo base
                toca_file_filename = "_".join(partes_nombre)
                
                # Limpiar y crear el nombre de archivo final
                filename = slugify(toca_file_filename) + '.pdf'
                
                # --- 2. Descargar y Guardar (Solo si el nombre fue exitoso) ---
                if not os.path.exists(filename):
                    # toca_id ya se obtuvo y validó arriba
                    toca_file_response = requests.get(f"{api_base_url}/toca/{toca_id}/file")
                    toca_file_response.raise_for_status() # Asegurarse que la descarga del PDF fue exitosa
                    
                    # Guardar el archivo
                    with open(filename, 'wb') as f:
                        f.write(toca_file_response.content)
                    print(f"Descargado: {filename}")
                    
                else:
                    print(f"Archivo ya existe: {filename}, se omite descarga.")
            
            except (TypeError, KeyError, AttributeError) as e_data:
                # Captura errores si 'materia_data' es None, o 'nombre' no existe, etc.
                print(f"Error de metadatos procesando Toca ID {toca.get('id', 'N/A')}: {e_data}. Se omite.")
            except requests.RequestException as e_req:
                # Captura errores de red (404, 500) al descargar el PDF
                print(f"Error de red al descargar Toca ID {toca.get('id', 'N/A')}: {e_req}. Se omite.")
            except Exception as e_other:
                # Captura cualquier otro error inesperado
                print(f"Error inesperado procesando Toca ID {toca.get('id', 'N/A')}: {e_other}. Se omite.")

    print("\n--- Proceso de descarga completado ---")

def main():
    # --- 1. Configuración ---
    # Ruta de destino para las sentencias (basada en tu celda de "manual_path")
    TARGET_FOLDER = '/home/nacho/Documents/diplomado_ia_cic/modulo_llm_api_tarea/prueba_dataset'
    API_BASE_URL = "https://publica-sentencias-backend.stjjalisco.gob.mx"
    PAGINATION_URL = f"{API_BASE_URL}/tocas?page=1&sala_id=&numero=&periodo=&derivado=&delito_id=&materia_id=&nombre_delito=&accion_especifica_id=&accion_simultanea_id="

    # --- 2. Preparar Directorio ---
    if not setup_directory(TARGET_FOLDER):
        return # Salir si no se puede configurar el directorio

    # --- 3. Obtener Paginación ---
    last_page = get_total_pages(PAGINATION_URL)
    if last_page is None:
        return # Salir si no se puede obtener la paginación

    # --- 4. Iniciar Descarga ---
    download_sentencias(last_page, API_BASE_URL)

if __name__ == "__main__":
    #main()



In [49]:
# FASE 1, 2 y 3: SCRIPT UNIFICADO DE CO-PILOTO JURÍDICO (v9.8)
# ---------------------------------------------------------------------------------------

# 1. Importación de Librerías y Configuración
import os
import re
import numpy as np
import time
import json
import chromadb # pip install chromadb
from sklearn.metrics.pairwise import cosine_similarity # pip install scikit-learn
import torch 
from transformers import AutoTokenizer, AutoModel # pip install transformers sentence-transformers
import requests # pip install requests
import spacy # pip install spacy && python -m spacy download es_core_news_sm
import pandas as pd
import PyPDF2 # pip install -U pypdf2
import glob
import math
import pyarrow as pa
import pyarrow.parquet as pq
from concurrent.futures import ProcessPoolExecutor, as_completed # <-- LIBRERÍA PARA PARALELIZACIÓN

try:
    from IPython.display import display, Markdown
except ImportError:
    def display(*args): pass
    def Markdown(text): return text

# --- CONFIGURACIÓN FINAL (CENTRALIZADA) ---
DB_PATH = "./chroma_db_juridica_7" 
EMBEDDING_MODEL_NAME = 'sentence-transformers/all-mpnet-base-v2' 
OLLAMA_API_URL = "http://localhost:11434/api/generate"
LOCAL_LLM_MODEL = "llama3.1:8b" 

# --- Configuración FASE 1 (Procesamiento de PDFs) ---
PDF_DATABASE_PATH = "/home/nacho/Documents/diplomado_ia_cic/modulo_llm_api_tarea/dataset_stjjalisco_sentencias" 
PDF_BATCH_SIZE = 500 
BATCH_TEMP_DIR = "./pdf_batches_temp_7"
PDF_MAX_PAGES_FULL_READ = 50
PDF_TARGET_TOTAL_PAGES_SAMPLE = 40
PDF_FIXED_START_PAGES = 10
PDF_FIXED_END_PAGES = 15
# Número de procesos paralelos para la Fase 1 (usa 'None' para usar todos los núcleos disponibles)
MAX_WORKERS_FASE_1 = None 

# --- Configuración FASE 2 (Indexación) ---
INDEXING_BATCH_SIZE = 50 

# --- Configuración FASE 3 (RAG y Clasificación) ---
RAG_NUM_RESULTS_DESIRED = 5         
RAG_NUM_RESULTS_TO_FETCH = 50       
TOP_N_CANDIDATES_INITIAL = 20
TOP_N_CANDIDATES_FINAL = 10 

# Pesos para Re-ranking Híbrido (Fallback Numérico)
COSINE_WEIGHT = 0.20  
JACCARD_WEIGHT = 0.40  
KEM_WEIGHT = 0.30      
CONCISION_WEIGHT = 0.10 
IRRELEVANT_TOKENS_PENALTY_FACTOR = 0.70 

# Parámetros para Penalización de Coherencia
KEM_THRESHOLD_FOR_PENALTY = 0.6  
MAX_TOKENS_FOR_PENALTY = 5       
IRRELEVANT_TOKENS = {'desmantelamiento', 'falsedad', 'informe', 'violacion', 'medica', 'medico'} 

# Lista de categorías base concisas para Inclusión Forzada
FORCED_CANDIDATES = [
    'robo-calificado',
    'lesiones-dolosas',
    'homicidio-culposo',
    'fraude-generico'
]

CACHE_FILE_CATEGORIES = 'rag_categories_cache7.json'
CACHE_FILE_EMBEDDINGS = 'rag_embeddings_cache7.npy'

# Parámetros del LLM
LLM_TEMPERATURE = 0.0
LLM_MAX_TOKENS = 1024

# Palabras clave para detectar fallos del LLM en QC
FAILURE_KEYWORDS = [
    "lo siento", "no puedo", "plantilla general", "sin hechos", "hechos extraídos que no",
    "hipotética", "asistencia legal", "no proporcionar", "información específica ni contexto", 
    "no tengo información", "no puedo analizar" 
]

# --- Variables Globales ---
_embedding_tokenizer, _embedding_model, _embedding_device = None, None, None
_real_categories_embeddings = None
nlp = None

In [46]:
# 2. DEFINICIÓN DE TODOS LOS PROMPTS (COMPLETO)
# ---------------------------------------------------------------------------------------

PROMPT_SINTESIS_FINAL = """
Eres un Abogado Coordinador experto. Tu trabajo es revisar los informes de análisis de precedentes que prepararon tus asistentes y sintetizarlos en un solo informe final.

**BASÁNDOTE ÚNICA Y EXCLUSIVAMENTE en los informes proporcionados**, responde la Consulta Original del Usuario. NO inventes información ni uses conocimiento externo.

Consulta Original del Usuario:
{text_consulta}

---
INFORMES DE PRECEDENTES (PARA ANALIZAR):
{informes_precedentes}
---

Formato de Salida Requerido (Markdown):

## Síntesis de Precedentes Relevantes

### 1. Resumen Ejecutivo (Síntesis)
[Escribe un párrafo que resuma la conclusión general de los informes, respondiendo directamente a la consulta del usuario SÓLO con datos de los informes.]

### 2. Precedente Más Relevante
[Identifica cuál de los informes es el más relevante y explica por qué.]

### 3. Análisis Comparativo
* **Puntos en Común:** [Menciona los argumentos o fallos que se repiten en los informes.]
* **Diferencias y Matices:** [Menciona si hay contradicciones o detalles únicos en cada informe.]

### 4. Conclusión para el Usuario
[Ofrece una conclusión final de 1-2 frases basada SÓLO en la evidencia combinada de los informes.]
"""

PROMPT_VERIFICACION_POST_REPORTE = """
Tu única tarea es actuar como un verificador de CALIDAD. 
Compara la Consulta Original del Usuario con el Informe de Análisis que se acaba de generar.
Debes responder 'SI' solo si el informe es 100% relevante para la consulta.

Responde 'NO' si el informe:
a) Habla de un tema legal completamente diferente (ej. consulta sobre "lesiones" y el informe habla de "fraude" o "violencia familiar").
b) Contiene alucinaciones o información que no tiene relación (ej. menciona "falta de pago" en un caso penal).

Sigue estos pasos para tu respuesta:
1.  **Análisis de Relevancia:** ¿El informe responde al contexto de la consulta?
2.  **Análisis de Alucinación:** ¿El informe contiene temas extraños no relacionados?
3.  **Veredicto:** En una línea nueva, escribe 'SI' o 'NO'.

**Consulta Original del Usuario:**
"{texto_consulta}"

---
**Informe de Análisis Generado:**
"{informe_generado}"
---

**Tu Análisis:**
"""


PROMPT_VERIFICACION_RELEVANCIA = """
Eres un Asistente de Verificación de Relevancia legal. Tu única tarea es ser 
extremadamente estricto y literal.

Compara el **Contexto Específico** de la Consulta del Usuario con el 
**Contexto Específico** del Documento Recuperado.

Responde 'SI' ÚNICAMENTE si el documento trata sobre el MISMO tema Y 
el MISMO contexto.

Responde 'NO' si los contextos son diferentes, aunque compartan palabras clave.
**EJEMPLO IMPORTANTE:** Si la consulta es sobre "lesiones en una riña de bar" 
(contexto: pelea pública) y el documento es sobre "lesiones por violencia familiar" 
(contexto: doméstico), DEBES RESPONDER 'NO'.

**Consulta del Usuario:**
"{texto_consulta}"

**Fragmento del Documento Recuperado:**
"{fragmento_documento}"

Sigue estos pasos para tu respuesta:
1.  **Análisis de Contexto de Consulta:** [Describe el tema y contexto de la consulta en 1 frase]
2.  **Análisis de Contexto de Documento:** [Describe el tema y contexto del documento en 1 frase]
3.  **Veredicto (SI/NO):** [Escribe tu veredicto final en una línea nueva]
"""


PROMPT_EXTRACCION_ENTIDADES = """
Tu única tarea es analizar la Consulta del Usuario y extraer de 1 a 3 conceptos jurídicos clave que sirvan como etiquetas de búsqueda. 
Es FUNDAMENTAL que respondas ÚNICAMENTE con los conceptos separados por comas, sin ningún tipo de explicación, preámbulo, disculpa o texto adicional. Si no puedes extraer, responde solo: 'FALLA'.
Consulta del Usuario: "{texto_consulta}"
Conceptos Clave:
"""

PROMPT_CLASIFICAR_MATERIA = """
Tu única tarea es clasificar el siguiente texto de una sentencia legal en una de las cuatro categorías: PENAL, CIVIL, MERCANTIL, o FAMILIAR.
Responde *únicamente* con la palabra de la categoría.

Contexto: {texto_contenido}
Respuesta:
"""

# --- PROMPTS DE EXTRACCIÓN (PASO 1: Estructuración Exhaustiva de Hechos) ---
# (Se incluyen las versiones optimizadas con [DATO NO DISPONIBLE] para robustez)

# --- PROMPTS DE EXTRACCIÓN (PASO 1) ---
# ACTUALIZADOS PARA IGNORAR TEXTO ANONIMIZADO

PROMPT_EXTRACCION_PENAL = """
Eres un Analista Legal Penal. Tu misión es extraer la información más exhaustiva y crítica de la SENTENCIA PENAL.

**NOTA IMPORTANTE: El texto puede contener datos anonimizados como '[ELIMINADO]' o '[No.1]...'. 
IGNORA esas marcas. Si un dato no se puede encontrar (ya sea porque fue eliminado o no se menciona), 
DEBES escribir [DATO NO DISPONIBLE] en su lugar.**

**Formato de Extracción (Detallado):**
- **Tribunal:** [Nombre completo de la Sala/Juzgado que resolvió, incluyendo la fecha exacta de la resolución]
- **Identificador Judicial:** [Toca/Expediente/Causa, tal como aparece]
- **Etapa Procesal:** [Ej: Sentencia Definitiva de Segunda Instancia, Auto de Vinculación a Proceso]
- **Delito(s) Principal(es):** [Lista completa y precisa de los delitos imputados/acreditados (ej: Robo Calificado Art. 233)]
- **Acusado(s):** [Nombre(s) o identificador(es) del acusado/sentenciado.]
- **Víctima(s)/Ofendido(s):** [Nombre(s) o identificador(es) de la víctima/ofendido.]
- **Hechos Clave:** [Narrativa breve y fáctica de la acción delictiva, las circunstancias de modo, tiempo y lugar.]
- **Decisión Específica:** [Fallo exacto: Se MODIFICA/CONFIRMA/REVOCA la sentencia, o la pena final impuesta (años/multa).]
- **Fundamento Legal:** [Enumera los artículos, códigos y jurisprudencias exactas citados como base de la decisión.]
- **Pruebas Decisivas:** [Enumera las pruebas cruciales que sustentaron el fallo (ej: Testimonial del ofendido, Peritaje balístico).]

Contexto (Sentencia):
{contexto}

Hechos Extraídos:
"""

PROMPT_EXTRACCION_CIVIL = """
Eres un Analista Legal Civil. Tu misión es extraer la información más exhaustiva y crítica de la SENTENCIA CIVIL.

**NOTA IMPORTANTE: El texto puede contener datos anonimizados como '[ELIMINADO]' o '[No.1]...'. 
IGNORA esas marcas. Si un dato no se puede encontrar (ya sea porque fue eliminado o no se menciona), 
DEBES escribir [DATO NO DISPONIBLE] en su lugar.**

**Formato de Extracción (Detallado):**
- **Tribunal:** [Nombre completo del Juzgado/Sala que resolvió, incluyendo la fecha exacta de la resolución]
- **Identificador Judicial:** [Número de Expediente o Toca, tal como aparece]
- **Etapa Procesal:** [Ej: Sentencia Definitiva de Segunda Instancia, Interlocutoria]
- **Acción Civil Principal:** [Acción ejercida de forma completa (ej. Acción Reivindicatoria, Rescisión de Contrato de Arrendamiento).]
- **Parte Actora:** [Nombre completo o identificador de quien demandó]
- **Parte Demandada:** [Nombre completo o identificador de quien fue demandado]
- **Objeto del Litigio:** [Describe el bien o derecho específico en disputa (ej. Inmueble con clave catastral X).]
- **Hechos Clave:** [Narrativa breve y fáctica de la disputa que llevó a la demanda (ej: Vicios ocultos en inmueble, Negligencia vehicular).]
- **Decisión Específica:** [Fallo exacto: Procede/No procede la acción, Condena/Absuelve a la parte demandada, Mención de la cuantía económica principal.]
- **Fundamento Legal:** [Enumera los artículos, códigos y jurisprudencias exactas citados como base de la decisión.]
- **Pruebas Decisivas:** [Enumera las pruebas documentales, periciales o testimoniales que fueron clave para la decisión.]

Contexto (Sentencia):
{contexto}

Hechos Extraídos:
"""

PROMPT_EXTRACCION_FAMILIAR = """
Eres un Analista Legal Familiar. Tu misión es extraer la información más exhaustiva y crítica de la SENTENCIA FAMILIAR.

**NOTA IMPORTANTE: El texto puede contener datos anonimizados como '[ELIMINADO]' o '[No.1]...'. 
IGNORA esas marcas. Si un dato no se puede encontrar (ya sea porque fue eliminado o no se menciona), 
DEBES escribir [DATO NO DISPONIBLE] en su lugar.**

**Formato de Extracción (Detallado):**
- **Tribunal:** [Nombre completo del Juzgado/Sala Familiar que resolvió, incluyendo la fecha exacta de la resolución]
- **Identificador Judicial:** [Número de Expediente]
- **Etapa Procesal:** [Ej: Sentencia Definitiva de Primera Instancia, Resolución de Incidente de Modificación]
- **Materia Principal:** [Tema Familiar específico (ej. Divorcio Necesario, Modificación de Guarda y Custodia).]
- **Progenitor Actor:** [Nombre de quien demanda]
- **Progenitor Demandado:** [Nombre del demandado]
- **Menores Involucrados:** [Lista de nombres o identificadores de los hijos menores de edad.]
- **Hechos Clave:** [Narrativa breve de la situación familiar que generó el conflicto.]
- **Decisión Específica:** [Fallo exacto sobre Custodia, Pensión (monto o porcentaje) y Régimen de Convivencias.]
- **Fundamento Clave:** [Describe el razonamiento clave del juez para proteger al menor, citando artículos de leyes familiares.]
- **Pruebas Decisivas:** [Enumera las pruebas cruciales: Estudios Socioeconómicos, Dictámenes Psicológicos, Opinión del Menor, etc.]

Contexto (Sentencia):
{contexto}

Hechos Extraídos:
"""

PROMPT_EXTRACCION_MERCANTIL = """
Eres un Analista Legal Mercantil. Tu misión es extraer la información más exhaustiva y crítica de la RESOLUCIÓN MERCANTIL.

**NOTA IMPORTANTE: El texto puede contener datos anonimizados como '[ELIMINADO]' o '[No.1]...'. 
IGNORA esas marcas. Si un dato no se puede encontrar (ya sea porque fue eliminado o no se menciona), 
DEBES escribir [DATO NO DISPONIBLE] en su lugar.**

**Formato de Extracción (Detallado):**
- **Tribunal:** [Nombre completo del Juzgado/Sala que resolvió, incluyendo la fecha exacta de la resolución]
- **Identificador Judicial:** [Número de Expediente/Toca]
- **Etapa Procesal:** [Ej: Sentencia Interlocutoria, Juicio Ejecutivo Mercantil]
- **Acción Principal:** [Materia Mercantil específica (ej. "Acción Cambiaria Directa", "Cumplimiento de Contrato").]
- **Parte Actora (Acreedora):** [Nombre de quien demanda el pago]
- **Parte Demandada (Deudora):** [Nombre de quien debe]
- **Documento Base:** [Tipo de documento que originó la deuda (ej. Pagaré N° X, Cheque Devuelto, Contrato de Suministro).]
- **Hechos Clave:** [Narrativa breve de la disputa, el monto de la deuda original y la razón del incumplimiento.]
- **Decisión Específica:** [Fallo exacto: Condena al pago, Absolución, Se declara la disolución de la sociedad.]
- **Consecuencias Financieras:** [Monto exacto a pagar (suerte principal), intereses (ordinarios/moratorios) y condena en costas.]
- **Fundamento Legal:** [Enumera los artículos, códigos (LGTOC, Código de Comercio) y jurisprudencias citados como base del fallo.]
- **Pruebas Decisivas:** [Enumera las pruebas cruciales (ej. El pagaré endosado, Certificación de Estado de Cuenta, Peritaje Contable).]

Contexto (Sentencia):
{contexto}

Hechos Extraídos:
"""

PROMPT_EXTRACCION_DEFAULT = """
Tu tarea es analizar el siguiente texto de una sentencia judicial y extraer los hechos clave de forma estructurada. 

**NOTA IMPORTANTE: El texto puede contener datos anonimizados como '[ELIMINADO]' o '[No.1]...'. 
IGNORA esas marcas. Si un dato no se puede encontrar (ya sea porque fue eliminado o no se menciona), 
DEBES escribir [DATO NO DISPONIBLE] en su lugar.**

**Formato de Extracción:**
- **Tribunal:** [Fecha y nombre completo del Tribunal/Juzgado]
- **Expediente:** [Número de Expediente/Toca]
- **Materia:** [Tema principal del litigio]
- **Partes:** [Identificación de las partes si se mencionan]
- **Hechos Clave:** [Narrativa breve de lo que ocurrió]
- **Decisión Específica:** [Fallo principal (ej. "Se condena", "Se absuelve", "Se modifica")]
- **Fundamentos/Pruebas:** [Razón principal de la decisión o pruebas clave, si se mencionan]
- **Leyes Citadas:** [Artículos de ley mencionados, si los hay]

Contexto (Sentencia):
{contexto}

Hechos Extraídos:
"""


# --- PROMPTS DE FALLBACK (CORREGIDOS) ---

PROMPT_EXTRACCION_GENERICA_FALLBACK = """
Tu única tarea es leer el documento legal y extraer el resumen más conciso de la siguiente información, **IGNORANDO cualquier marca de anonimización** ([ELIMINADO], [No.X]):
1.  **Materia y Asunto:** El tema principal del documento (ej. Divorcio, Robo, Demanda de Pago).
2.  **Partes:** Quién es el acusado/demandante y quién es la víctima/demandado.
3.  **Fallo:** La decisión final (ej. Se absuelve, Se ordena reponer el procedimiento).

Responde en formato de lista de puntos, **utilizando SOLO la información disponible**. Si un punto no está claro, escribe 'Dato Confuso o No Encontrado'.

Contexto (Sentencia):
{contexto}

Resumen de Extracción Genérica:
"""

PROMPT_REPORTE_GENERICO_FALLBACK = """
Utiliza el siguiente resumen de hechos (extraído por un proceso genérico) para generar un informe FINAL de análisis conciso.
No intentes rellenar los campos faltantes, simplemente reporta lo que se extrajo.

**Resumen de Extracción Genérica:**
{hechos}

**Formato del Informe:**
## Informe de Análisis (Extracción de Fallback)

### 1. Resumen de Hechos y Decisión
[Sintetiza la información del 'Resumen de Extracción Genérica' en un solo párrafo, enfocándote en el fallo y el tema principal.]

### 2. Estado del Proceso
[Reporta la información de Materia, Partes y el Fallo final según la extracción genérica.]
"""

# --- PROMPTS DE REPORTE (PASO 2: Optimizados con Análisis y Razonamiento) ---

PROMPT_REPORTE_PENAL = """
Con base en los siguientes HECHOS EXTRAÍDOS de una sentencia penal, genera un informe legal profesional, **detallado y analítico**. 
Utiliza los 'Hechos Extraídos' para construir una narrativa coherente y explicar las razones del tribunal.

**Hechos Extraídos:**
{hechos}

**Formato del Informe:**
## Informe de Análisis Penal
### 1. Resumen Ejecutivo
[Escribe un resumen breve (2-3 frases) que describa el delito principal y cuál fue la decisión final del tribunal sobre el acusado.]

### 2. Datos de Identificación del Proceso
- **Tribunal:** [Pega aquí el dato de 'Tribunal' de los Hechos Extraídos]
- **Expediente:** [Pega aquí el dato de 'Número de Expediente' de los Hechos Extraídos]
- **Etapa Procesal:** [Pega aquí el dato de 'Etapa Procesal' de los Hechos Extraídos]
- **Delito(s):** [Pega aquí el dato de 'Delito(s) Principal(es)' de los Hechos Extraídos]

### 3. Partes Involucradas
- **Acusado(s):** [Pega aquí el dato de 'Acusado(s)' de los Hechos Extraídos]
- **Víctima(s)/Ofendido(s):** [Pega aquí el dato de 'Víctima(s)/Ofendido(s)' de los Hechos Extraídos]

### 4. Análisis de Hechos y Decisión
**Hechos Clave:**
[Pega aquí la narrativa de 'Hechos Clave' de los Hechos Extraídos]

**Decisión del Tribunal:**
[Pega aquí la 'Decisión Específica' de los Hechos Extraídos]

**Razonamiento del Tribunal (Análisis):**
[Basado en los 'Fundamentos Legales' y 'Pruebas Decisivas' de los Hechos Extraídos, **explica** por qué el tribunal tomó esa decisión. ¿Qué pruebas fueron más importantes? ¿Cómo se aplicaron los artículos citados?]
"""

PROMPT_REPORTE_CIVIL = """
Con base en los siguientes HECHOS EXTRAÍDOS de una sentencia civil, genera un informe legal profesional, **detallado y analítico**. 
Utiliza los 'Hechos Extraídos' para construir una narrativa coherente y explicar las razones del tribunal.

**Hechos Extraídos:**
{hechos}

**Formato del Informe:**
## Informe de Análisis Civil
### 1. Resumen Ejecutivo
[Escribe un resumen breve (2-3 frases) que describa la acción principal, el objeto del litigio y quién ganó el juicio.]

### 2. Datos de Identificación del Proceso
- **Tribunal:** [Pega aquí el dato de 'Tribunal' de los Hechos Extraídos]
- **Expediente:** [Pega aquí el dato de 'Identificador Judicial' de los Hechos Extraídos]
- **Etapa Procesal:** [Pega aquí el dato de 'Etapa Procesal' de los Hechos Extraídos]
- **Acción Principal:** [Pega aquí el dato de 'Acción Civil Principal' de los Hechos Extraídos]

### 3. Partes y Objeto del Litigio
- **Parte Actora:** [Pega aquí el dato de 'Parte Actora' de los Hechos Extraídos]
- **Parte Demandada:** [Pega aquí el dato de 'Parte Demandada' de los Hechos Extraídos]
- **Objeto del Litigio:** [Pega aquí el dato de 'Objeto del Litigio' de los Hechos Extraídos]

### 4. Análisis de Hechos y Decisión
**Hechos Clave:**
[Pega aquí la narrativa de 'Hechos Clave' de los Hechos Extraídos]

**Decisión del Tribunal:**
[Pega aquí la 'Decisión Específica' de los Hechos Extraídos]

**Razonamiento del Tribunal (Análisis):**
[Basado en los 'Fundamentos Legales' y 'Pruebas Decisivas', **explica** por qué la acción procedió o no. ¿Qué parte probó sus afirmaciones y cómo?]
"""

PROMPT_REPORTE_FAMILIAR = """
Con base en los siguientes HECHOS EXTRAÍDOS de una sentencia familiar, genera un informe legal profesional, **detallado y analítico**. 
Utiliza los 'Hechos Extraídos' para construir una narrativa coherente, **enfocándote en el Interés Superior del Menor**.

**Hechos Extraídos:**
{hechos}

**Formato del Informe:**
## Informe de Análisis Familiar
### 1. Resumen Ejecutivo
[Escribe un resumen breve (2-3 frases) que describa el conflicto familiar y la decisión final sobre los menores (custodia, alimentos).]

### 2. Datos de Identificación del Proceso
- **Tribunal:** [Pega aquí el dato de 'Tribunal' de los Hechos Extraídos]
- **Expediente:** [Pega aquí el dato de 'Identificador Judicial' de los Hechos Extraídos]
- **Etapa Procesal:** [Pega aquí el dato de 'Etapa Procesal' de los Hechos Extraídos]
- **Materia Principal:** [Pega aquí el dato de 'Materia Principal' de los Hechos Extraídos]

### 3. Partes y Menores Involucrados
- **Progenitor Actor:** [Pega aquí el dato de 'Progenitor Actor' de los Hechos Extraídos]
- **Progenitor Demandado:** [Pega aquí el dato de 'Progenitor Demandado' de los Hechos Extraídos]
- **Menores:** [Pega aquí el dato de 'Menores Involucrados' de los Hechos Extraídos]

### 4. Decisión y Fundamentos (Enfoque en Menores)
**Decisión Específica:**
[Pega aquí la 'Decisión Específica' de los Hechos Extraídos (Custodia, Pensión, Convivencia).]

**Razonamiento del Tribunal (Análisis):**
[Basado en el 'Fundamento Clave' y las 'Pruebas Decisivas', **explica** por qué el juez tomó esa decisión, centrándote en cómo se protegió el Interés Superior del Menor (ej. "Se otorgó la custodia a la madre basado en los estudios psicológicos que indicaban...").]
"""

PROMPT_REPORTE_MERCANTIL = """
Con base en los siguientes HECHOS EXTRAÍDOS de una resolución mercantil, genera un informe legal profesional, **detallado y analítico**. 
Utiliza los 'Hechos Extraídos' para construir una narrativa coherente y explicar las consecuencias económicas.

**Hechos Extraídos:**
{hechos}

**Formato del Informe:**
## Informe de Análisis Mercantil
### 1. Resumen Ejecutivo
[Escribe un resumen breve (2-3 frases) que describa la acción mercantil, el documento base y el resultado (si se ordenó el pago o no).]

### 2. Datos del Proceso
* **Tribunal:** [Pega aquí el dato de 'Tribunal' de los Hechos Extraídos]
* **Expediente:** [Pega aquí el dato de 'Identificador Judicial' de los Hechos Extraídos]
* **Etapa Procesal:** [Pega aquí el dato de 'Etapa Procesal' de los Hechos Extraídos]
* **Acción Principal:** [Pega aquí el dato de 'Acción Principal' de los Hechos Extraídos]

### 3. Partes y Documento Base
* **Parte Actora (Acreedora):** [Pega aquí el dato de 'Parte Actora (Acreedora)' de los Hechos Extraídos]
* **Parte Demandada (Deudora):** [Pega aquí el dato de 'Parte Demandada (Deudora)' de los Hechos Extraídos]
* **Documento Base:** [Pega aquí el dato de 'Documento Base' de los Hechos Extraídos]

### 4. Hechos y Decisión
**Hechos Clave:**
[Pega aquí la narrativa de 'Hechos Clave' de los Hechos Extraídos]

**Decisión Final:**
[Pega aquí la 'Decisión Específica' de los Hechos Extraídos]

### 5. Análisis y Consecuencias
**Razonamiento del Tribunal (Análisis):**
[Basado en los 'Fundamentos Legales' y 'Pruebas Decisivas', **explica** por qué se condenó o absolvió al deudor (ej. "El pagaré cumplió los requisitos...", "La excepción de prescripción fue procedente...").]

**Consecuencias Financieras:**
[Pega aquí el dato de 'Consecuencias Financieras' de los Hechos Extraídos (Suerte principal, intereses, costas).]
"""

PROMPT_REPORTE_DEFAULT = """
Con base en los siguientes HECHOS EXTRAÍDOS, genera un informe legal estructurado y **analítico**.
Utiliza los 'Hechos Extraídos' para construir una narrativa coherente.

**Hechos Extraídos:**
{hechos}

**Formato del Informe:**
## Informe General de la Resolución
### 1. Resumen de la Decisión
[Escribe un resumen breve del tema principal y la decisión final.]

### 2. Detalles del Caso
* **Materia:** [Pega aquí el dato de 'Materia' de los Hechos Extraídos]
* **Tribunal:** [Pega aquí el dato de 'Tribunal' de los Hechos Extraídos]
* **Expediente:** [Pega aquí el dato de 'Expediente' de los Hechos Extraídos]
* **Partes:** [Pega aquí el dato de 'Partes' de los Hechos Extraídos]

### 3. Hechos y Decisión
**Hechos Clave:**
[Pega aquí la narrativa de 'Hechos Clave' de los Hechos Extraídos]

**Decisión Final:**
[Pega aquí la 'Decisión Específica' de los Hechos Extraídos]

### 4. Fundamentos del Tribunal
[**Explica** brevemente por qué el tribunal tomó esa decisión, basándote en los 'Fundamentos/Pruebas' y 'Leyes Citadas'.]
"""

In [3]:
# 3. FUNCIONES AUXILIARES (EMBEDDINGS, LLM, SPACY, MÉTRICAS)
# ---------------------------------------------------------------------------------------

def get_embedding_model_and_tokenizer():
    """Carga y prepara el modelo de embeddings (all-mpnet-base-v2)."""
    global _embedding_tokenizer, _embedding_model, _embedding_device
    if _embedding_tokenizer is None:
        try:
            print("Cargando modelo de embeddings...")
            _embedding_tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
            _embedding_model = AutoModel.from_pretrained(EMBEDDING_MODEL_NAME)
            _embedding_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            _embedding_model.to(_embedding_device)
            print(f"Modelo en dispositivo: {_embedding_device}")
            return True
        except Exception as e:
            print(f"ERROR FATAL al cargar modelo de embeddings: {e}")
            return False
    return True

def generate_embeddings(texts):
    """Genera embeddings vectoriales (768 dimensiones) para el texto."""
    if not get_embedding_model_and_tokenizer(): return None
    if isinstance(texts, str): texts = [texts]
    if not all(isinstance(t, str) for t in texts): return None
        
    tokenizer, model, device = _embedding_tokenizer, _embedding_model, _embedding_device
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

def call_local_llm(prompt):
    """Realiza la llamada a la API local de Ollama (LLM) con opciones de tuning."""
    payload = {
        "model": LOCAL_LLM_MODEL, 
        "prompt": prompt, 
        "stream": False, 
        "options": {
            "temperature": LLM_TEMPERATURE, 
            "num_predict": LLM_MAX_TOKENS
        }
    }
    try:
        response = requests.post(OLLAMA_API_URL, json=payload, timeout=600)
        response.raise_for_status()
        raw_response = response.json().get('response', '').strip()
        
        # Limpieza robusta de formato LLM
        cleaned_response = re.sub(r'^\d+\.\s*|^\-\s*', '', raw_response)
        return cleaned_response.lower().replace("*", "").replace("\"", "").strip()
    except requests.exceptions.RequestException as e:
        print(f"ERROR: No se pudo conectar con el LLM en {OLLAMA_API_URL}: {e}")
        return None

def initialize_spacy_model():
    global nlp
    if nlp is None:
        print("Cargando modelo de SpaCy (es_core_news_sm)...")
        try:
            nlp = spacy.load("es_core_news_sm")
            print("✅ Modelo SpaCy cargado.")
            return True
        except OSError:
            print("\n" + "="*60)
            print("❌ ERROR CRÍTICO: Modelo 'es_core_news_sm' de SpaCy no encontrado.")
            print("   Por favor, instálalo ejecutando en tu terminal: python -m spacy download es_core_news_sm")
            print("="*60)
            return False
    return True

def get_lemmatized_tokens(text):
    if nlp is None: return set()
    clean_text = text.lower().replace('-', ' ') 
    doc = nlp(clean_text)
    return {token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space and token.is_alpha}

def calculate_jaccard_similarity(set1, set2):
    if not set1 and not set2: return 0.0
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

def calculate_key_entity_match(category_tokens, entity_tokens, high_importance_tokens):
    """
    MÉTRICA KEM: Recompensa la presencia de tokens de alta importancia (dinámómicos).
    """
    relevant_important_tokens = high_importance_tokens.intersection(entity_tokens)
    
    if not relevant_important_tokens:
        return 0.0
    
    important_intersection = relevant_important_tokens.intersection(category_tokens)
    
    return len(important_intersection) / len(relevant_important_tokens)

def extract_key_entities(query_text):
    """Extrae entidades clave usando el LLM con lógica de Control de Calidad (QC)."""
    print("\n--- PASO 2a: EXTRACCIÓN DE ENTIDADES CLAVE ---")
    
    global PROMPT_EXTRACCION_ENTIDADES 
    
    prompt = PROMPT_EXTRACCION_ENTIDADES.format(texto_consulta=query_text)
    entities_str = call_local_llm(prompt)
    
    # --- CONTROL DE CALIDAD (QC) ---
    if not entities_str or "falla" in entities_str.lower().strip() or "no hay un texto" in entities_str.lower():
        print("   ⚠️ Fallo del LLM al extraer entidades. Usando la consulta completa como entidades clave.")
        return query_text # Devolvemos la consulta original como fallback
    
    print(f"   -> Entidades extraídas: '{entities_str}'")
    return entities_str

In [4]:
# --- Funciones FASE 1: Procesamiento de PDF ---

# Regex de Fase 1 (Ruido y Estructura)
RE_NOISE_F1 = re.compile(r"\n?\s*toca\s*[\d/]+.*?\n|"
                      r"\n?\s*exp(ediente)?:?\s*[\d/a-z-]+.*?\n|"
                      r"\n?\s*p[aá]gina\s*\d+.*?\n|"
                      r"\n?\s*--- PÁG...|--- PAGE... ---.*?\n|" # Mejorado
                      r"\n?\s*documento\s+para\s+versi[oó]n\s+electr[oó]nica.*?\n|"
                      r"\n?\s*el\s+documento\s+fue\s+testado\s+con\s+el\s+programa\s+'elida'.*?\n|"
                      r"\n?\s*supremo\s+tribunal\s+de\s+justicia\s+del\s+estado.*?\n|"
                      r"\n?\s*(primera|segunda|tercera|cuarta|quinta|sexta|s[eé]ptima|octava|novena|d[eé]cima)\s+sala.*?\n",
                      re.IGNORECASE | re.DOTALL)
                      
RE_STRUCTURAL_KEYWORDS_F1 = re.compile(r"^\s*"
                      r"("
                      r"V\s*I\s*S\s*T\s*O\s*(?:S)?\b"
                      r"|R\s*E\s*S\s*U\s*L\s*T\s*A\s*N\s*D\s*O\s*(?:S)?\b"
                      r"|A\s*N\s*T\s*E\s*C\s*E\s*D\s*E\s*N\s*T\s*E\s*(?:S)?\b"
                      r"|C\s*O\s*N\s*S\s*I\s*D\s*E\s*R\s*A\s*N\s*D\s*O\s*(?:S)?\b"
                      r"|R\s*E\s*S\s*U\s*E\s*L\s*V\s*E\b"
                      r"|P\s*R\s*O\s*P\s*O\s*S\s*I\s*C\s*I\s*O\s*N\s*(?:E\s*S)?\b"
                      r"|R\s*E\s*S\s*O\s*L\s*U\s*T\s*I\s*V\s*O\s*(?:S)?\b"
                      r"|SE\s+RESUELVE\b"
                      r")", re.IGNORECASE | re.MULTILINE)
                      
RE_FINAL_BOILERPLATE_F1 = re.compile(r"(\bfundamentaci[oó]n\s+legal\b|\bNOTIF[ÍI]QUESE\b|As[ií]\s+lo\s+resolvi[oó]|Arch[ií]vese)", re.IGNORECASE | re.DOTALL)

def extract_text_from_pdf(pdf_path):
    """Extrae texto de un PDF usando muestreo dinámico (Función de Fase 1)."""
    text = ""; filename = os.path.basename(pdf_path)
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file); total_pages = len(reader.pages)
            if total_pages == 0: 
                # print(f"  -> ADV: '{filename}' sin págs.") # Log silenciado
                return ""
            pages_to_read_indices = set(); strategy_log = ""
            
            if total_pages <= PDF_MAX_PAGES_FULL_READ:
                strategy_log = f"Completo ({total_pages} págs)"; pages_to_read_indices.update(range(total_pages))
            else:
                num_start=min(PDF_FIXED_START_PAGES,total_pages); num_end=min(PDF_FIXED_END_PAGES,max(0,total_pages-num_start))
                num_middle_target=max(0,PDF_TARGET_TOTAL_PAGES_SAMPLE-num_start-num_end); middle_start_index=num_start
                middle_end_index=total_pages-num_end; available_middle_pages=max(0,middle_end_index-middle_start_index)
                num_middle_actual=min(num_middle_target,available_middle_pages)
                strategy_log = (f"Muestreo ({total_pages}p -> Ini:{num_start}, Med:{num_middle_actual}, Fin:{num_end})")
                pages_to_read_indices.update(range(num_start)); pages_to_read_indices.update(range(total_pages - num_end, total_pages))
                if num_middle_actual > 0:
                    if middle_end_index > middle_start_index:
                         middle_indices_float = np.linspace(middle_start_index, middle_end_index - 1, num=num_middle_actual)
                         middle_indices_int = set(int(round(idx_float)) for idx_float in middle_indices_float) 
                         pages_to_read_indices.update(middle_indices_int)
            
            sorted_indices = sorted(list(pages_to_read_indices)); num_pages_read = len(sorted_indices)
            # print(f"    -> PDF Largo: {strategy_log}. Leyendo {num_pages_read} págs.") # Log silenciado
            extracted_texts=[]; last_read_index=-1
            
            for index in sorted_indices:
                if last_read_index!=-1 and index>last_read_index+1: omitted_count=index-(last_read_index+1); extracted_texts.append(f"\n\n [... {omitted_count} pág(s) omitida(s) ...] \n\n")
                try:
                    page=reader.pages[index]; page_text=page.extract_text() or ""; extracted_texts.append(f"\n--- PÁG {index + 1}/{total_pages} ---\n{page_text}"); last_read_index = index
                except Exception as page_e: 
                    print(f"    -> ERROR: Lectura pág {index + 1} de '{filename}': {page_e}"); extracted_texts.append(f"\n\n [... ERROR LEYENDO PÁG {index + 1} ...] \n\n"); last_read_index=index
            text="\n".join(extracted_texts)
    except PyPDF2.errors.PdfReadError: 
        print(f"  -> ERROR leer PDF '{filename}' (corrupto). Omitiendo."); return ""
    except Exception as e: 
        print(f"  -> ERROR inesperado PDF '{filename}': {e}. Omitiendo."); return ""
    return text

def extract_and_clean_legal_text(raw_text, filename=""):
    """Limpia el texto extraído y aísla el cuerpo central (Función de Fase 1)."""
    pre_filtered_text = RE_NOISE_F1.sub(" ", raw_text)
    matches = list(RE_STRUCTURAL_KEYWORDS_F1.finditer(pre_filtered_text))
    core_text = ""
    
    if matches:
        start_pos = matches[0].start()
        end_pos = -1
        
        last_section_match = matches[-1]
        search_start_after_section = last_section_match.end()
        
        boilerplate_match = RE_FINAL_BOILERPLATE_F1.search(pre_filtered_text, pos=search_start_after_section)
        
        if boilerplate_match:
            end_pos = boilerplate_match.start()
        else:
            end_pos = len(pre_filtered_text)

        core_text = pre_filtered_text[start_pos:end_pos]
    else:
        # print(f"    -> ADV: No se encontraron marcadores estructurales en '{filename}'.") # Log silenciado
        boilerplate_match = RE_FINAL_BOILERPLATE_F1.search(pre_filtered_text)
        if boilerplate_match: 
            core_text = pre_filtered_text[:boilerplate_match.start()]
        else: 
            core_text = pre_filtered_text

    cleaned_text = re.sub(r'\s+', ' ', core_text).strip()
    cleaned_text = cleaned_text.replace('\n', ' ').replace('\t', ' ')
    cleaned_text = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑüÜ.,;?¿!¡\s-]', '', cleaned_text) # Limpieza estricta
    
    return cleaned_text.lower().strip()

def parse_metadata_from_filename(filename):
    """Extrae metadatos (Materia, Acción) del nombre del archivo (Función de Fase 1)."""
    base_name = filename.replace(".pdf", "").lower()
    try:
        parts = base_name.split('_')
        if len(parts) < 3: return None
        materia_principal = parts[1].strip()
        delito_o_accion = '_'.join(parts[2:]).strip()
    except Exception:
        return None 
    if not materia_principal or not delito_o_accion:
        return None
    return {
        'materia_principal': materia_principal, 
        'delito_o_accion': delito_o_accion
    }

def process_single_pdf(pdf_path):
    """(Helper de Fase 1) Procesa un solo PDF para paralelización."""
    filename = os.path.basename(pdf_path)
    
    metadata = parse_metadata_from_filename(filename) 
    if not metadata:
        # print(f"    -> ADV: No se pudo parsear nombre de '{filename}'.") # Log silenciado
        return None # Devuelve None si el parseo falla

    final_materia = metadata['materia_principal']
    final_accion = metadata['delito_o_accion']
            
    extracted_raw_text = extract_text_from_pdf(pdf_path)
    if not extracted_raw_text: 
        return None # Devuelve None si la extracción de PDF falla

    cleaned_section_text = extract_and_clean_legal_text(extracted_raw_text, filename=filename)
    
    if not cleaned_section_text.strip() or len(cleaned_section_text) < 100: 
        # print(f"    -> ADV: Texto extraído/limpio demasiado corto (<100 chars) en '{filename}'.") # Log silenciado
        return None # Devuelve None si la limpieza falla
    
    return {'id': filename, 'texto': cleaned_section_text, 'materia_principal': final_materia, 'delito_o_accion': final_accion}

def fase1_procesar_pdfs(directory_path=PDF_DATABASE_PATH, batch_size=PDF_BATCH_SIZE, temp_dir=BATCH_TEMP_DIR):
    """(FASE 1) Escanea PDFs, extrae, limpia y guarda metadatos en lotes Parquet (PARALELIZADO)."""
    print(f"\n--- INICIANDO FASE 1: PROCESAMIENTO DE PDFS EN LOTES (PARALELIZADO) ---"); 
    if not os.path.isdir(directory_path): print(f"ERROR: Directorio PDFs '{directory_path}' no existe."); return []
    if not os.path.exists(temp_dir):
        try: os.makedirs(temp_dir); print(f"Dir temporal lotes creado: '{temp_dir}'")
        except OSError as e: print(f"ERROR: Creación dir temporal '{temp_dir}': {e}"); return []
    else: print(f"Dir temporal lotes: '{temp_dir}'")
    
    pdf_files = glob.glob(os.path.join(directory_path, '*.pdf'))
    if not pdf_files: print(f"No se encontraron PDFs en '{directory_path}'."); return []
    
    total_files = len(pdf_files); num_batches = math.ceil(total_files / batch_size); print(f"{total_files} PDFs encontrados. Procesando en {num_batches} lotes.")
    batch_files_paths = []; processed_files_count = 0; failed_files = 0
    
    for i in range(num_batches):
        start_index = i * batch_size; end_index = start_index + batch_size; batch_pdf_paths = pdf_files[start_index:end_index]
        print(f"\n--- Procesando Lote {i+1}/{num_batches} ({len(batch_pdf_paths)} archivos) con {MAX_WORKERS_FASE_1 or 'todos los'} núcleos ---")
        batch_data = []
        
        # --- INICIO DE PARALELIZACIÓN ---
        with ProcessPoolExecutor(max_workers=MAX_WORKERS_FASE_1) as executor:
            futures = {executor.submit(process_single_pdf, pdf_path): pdf_path for pdf_path in batch_pdf_paths}
            
            for future in as_completed(futures):
                result = future.result()
                if result:
                    batch_data.append(result)
                    processed_files_count += 1
                else:
                    failed_files += 1
        # --- FIN DE PARALELIZACIÓN ---
        
        if batch_data:
            batch_df = pd.DataFrame(batch_data); batch_filename = f"batch_{i+1:04d}.parquet"; batch_filepath = os.path.join(temp_dir, batch_filename)
            try: table = pa.Table.from_pandas(batch_df); pq.write_table(table, batch_filepath); batch_files_paths.append(batch_filepath); print(f"--- Lote {i+1} guardado: '{batch_filepath}' ({len(batch_data)} regs) ---")
            except Exception as e: print(f"ERROR guardando lote {i+1}: {e}")
        else: print(f"--- Lote {i+1} vacío. ---")
            
    print(f"\n--- Fin Procesamiento PDFs ---"); print(f"Procesados OK: {processed_files_count}/{total_files}"); print(f"Fallos (Nombre/Texto): {failed_files}"); print(f"Lotes generados: {len(batch_files_paths)} en '{temp_dir}'.")
    return batch_files_paths

In [5]:
# --- Funciones FASE 2: Indexación en RAG ---

def generate_embeddings_in_sub_batches(texts):
    """(FASE 2) Genera embeddings en sub-lotes para gestionar la memoria GPU."""
    all_embeddings = []
    num_sub_batches = math.ceil(len(texts) / INDEXING_BATCH_SIZE)
    print(f"Generando embeddings para {len(texts)} documentos en {num_sub_batches} sub-lotes de {INDEXING_BATCH_SIZE}...")

    for i in range(num_sub_batches):
        start_idx = i * INDEXING_BATCH_SIZE
        end_idx = min((i + 1) * INDEXING_BATCH_SIZE, len(texts))
        sub_batch_texts = texts[start_idx:end_idx]
        
        try:
            embeddings_batch = generate_embeddings(sub_batch_texts) 
            if embeddings_batch is None: raise Exception("Fallo en generate_embeddings")
            
            all_embeddings.append(embeddings_batch)
            print(f"   -> Sub-lote {i+1}/{num_sub_batches} procesado OK.")
        except Exception as e:
            if "CUDA out of memory" in str(e): 
                print(f"   ❌ ERROR CRÍTICO OOM en sub-lote {i+1}. Reduce el valor de INDEXING_BATCH_SIZE (actual: {INDEXING_BATCH_SIZE}).")
            else:
                 print(f"   ❌ ERROR en sub-lote {i+1}: {e}")
            raise e
            
    return np.vstack(all_embeddings)

def fase2_indexar_rag(db_path=DB_PATH, temp_dir=BATCH_TEMP_DIR):
    """
    (FASE 2) Carga Parquets, genera embeddings y los indexa en ChromaDB.
    (VERSIÓN SIMPLIFICADA: Se elimina la validación de coherencia de la IA para evitar OOM).
    """
    
    parquet_files = glob.glob(os.path.join(temp_dir, '*.parquet'))
    if not parquet_files: 
        print(f"ERROR CRÍTICO (Fase 2): No se encontraron archivos Parquet en '{temp_dir}'.")
        return None
        
    print(f"\n--- INICIANDO FASE 2: INDEXACIÓN (Versión Simplificada) ---");
    
    collection_name = "sentencias_judiciales"
    try:
        print(f"Inicializando cliente de ChromaDB en la ruta: '{db_path}'")
        client = chromadb.PersistentClient(path=db_path)
        
        collection_list = client.list_collections()
        collection_exists = any(c.name == collection_name for c in collection_list)

        if collection_exists:
            print(f"Detectada colección existente '{collection_name}'. Eliminando para indexación limpia...")
            client.delete_collection(name=collection_name)
            
        collection = client.get_or_create_collection(name=collection_name)
        print(f"DB y colección '{collection_name}' listas. Documentos actuales: {collection.count()}.")
        
    except Exception as e: 
        print(f"ERROR CRÍTICO al inicializar ChromaDB: {e}")
        return None
    
    if not get_embedding_model_and_tokenizer():
        print("Deteniendo indexación debido a error en el modelo de embeddings.")
        return None
        
    total_docs_processed = 0
    start_time = time.time()

    for i, file_path in enumerate(parquet_files):
        print(f"\n--- Procesando Lote de Archivos {i+1}/{len(parquet_files)}: {os.path.basename(file_path)} ---")
        try:
            df = pq.read_table(file_path).to_pandas()
            if df.empty:
                print("   -> Archivo Parquet vacío. Saltando.")
                continue
        except Exception as e:
            print(f"   -> ERROR al cargar el archivo Parquet: {e}. Saltando.")
            continue
        
        # --- VALIDACIÓN DE IA ELIMINADA ---
        # Ya no se llama a call_local_llm (PROMPT_CLASIFICAR_MATERIA) aquí.
        # Confiamos en los datos de la Fase 1.
        df_validado = df
        
        texts_to_embed = df_validado['texto'].tolist()
        
        try:
            embeddings = generate_embeddings_in_sub_batches(texts_to_embed)
        except Exception:
            print(f"   -> ERROR FATAL durante la generación de embeddings para este lote. Saltando.")
            continue
            
        metadatas_batch = [{'materia_principal': str(row['materia_principal']), 
                            'delito_o_accion': str(row['delito_o_accion'])} 
                           for _, row in df_validado.iterrows()]
        ids_batch = df_validado['id'].tolist()
        
        try: 
            collection.add(
                embeddings=embeddings.tolist(),
                documents=texts_to_embed,
                metadatas=metadatas_batch,
                ids=ids_batch
            )
            total_docs_processed += len(ids_batch)
            print(f"   -> ÉXITO: {len(ids_batch)} documentos de este lote fueron añadidos a ChromaDB.")
        except Exception as e: 
            print(f"   -> ERROR CRÍTICO al añadir el lote a ChromaDB: {e}")
            
    end_time = time.time()
    total_time = end_time - start_time
    
    print(f"\n--- FIN DE LA INDEXACIÓN ---")
    print(f"Proceso completado en {total_time:.2f} segundos.")
    print(f"Total de documentos añadidos: {total_docs_processed}")
    
    # ... (Verificación final de la colección) ...
    
    return collection

In [6]:
# --- Funciones FASE 3: Lógica del Co-Piloto (Clasificación y RAG) ---

def get_real_categories_from_db(collection, batch_size=5000):
    """(FASE 3) Obtiene categorías únicas (acción/delito) de la RAG."""
    print(f"-> Obteniendo categorías reales desde la base de datos RAG en lotes de {batch_size}...")
    categories_set = set()
    total_count = collection.count()
    offset = 0
    
    try:
        while offset < total_count:
            results = collection.get(
                offset=offset, 
                limit=batch_size, 
                include=["metadatas"]
            )
            
            metadata = results.get('metadatas')
            if not metadata:
                print(f"   ⚠️ Lote con offset {offset} no devolvió metadatos. Terminando búsqueda.")
                break
                
            for item in metadata:
                if item and 'delito_o_accion' in item:
                    accion = item['delito_o_accion']
                    if accion:
                        categories_set.add(accion) 
            
            offset += batch_size
            # print(f"   -> Procesados {min(offset, total_count)}/{total_count} documentos. Categorías únicas: {len(categories_set)}") # Log silenciado
        
        if not categories_set: 
            return []
            
        print(f"✅ Se encontraron {len(categories_set)} categorías únicas (Acción) en la RAG.")
        return sorted(list(categories_set))
        
    except Exception as e:
        print(f"ERROR al obtener metadatos de la RAG: {e}")
        return []

def classify_dynamically(query_text, real_categories, real_categories_embeddings):
    """Implementa la estrategia de Re-ranking Híbrido Numérico (v7), seleccionando el mejor score híbrido."""
    
    print("\n--- PASO 1: BÚSQUEDA SEMÁNTICA INICIAL (FILTRO GRUESO) ---")
    query_embedding = generate_embeddings([query_text])
    if query_embedding is None: return None

    # 1. GENERACIÓN DINÁMICA DE TOKENS Y SEMÁNTICA INICIAL
    key_entities_str = extract_key_entities(query_text)
    entity_tokens = get_lemmatized_tokens(key_entities_str) if key_entities_str else get_lemmatized_tokens(query_text)
    
    high_importance_tokens = entity_tokens
    print(f"   -> KEM Tokens Dinámicos: {list(high_importance_tokens)}") 
    
    cosine_similarities = cosine_similarity(query_embedding, real_categories_embeddings)[0]
    initial_top_indices = np.argsort(cosine_similarities)[-TOP_N_CANDIDATES_INITIAL:][::-1]
    
    # 2. CALCULAR SCORES
    candidates_with_scores = []
    
    for i in initial_top_indices:
        category_text = real_categories[i]
        category_tokens = get_lemmatized_tokens(category_text)
        
        # 2a. Cálculos de Métricas
        cosine_score = cosine_similarities[i]
        jaccard_score = calculate_jaccard_similarity(entity_tokens, category_tokens)
        kem_score = calculate_key_entity_match(category_tokens, entity_tokens, high_importance_tokens)
        num_tokens = max(1, len(category_tokens))
        concision_score = 1 / np.sqrt(num_tokens) if num_tokens > 0 else 0.0
        
        # 2b. Cálculo de Penalización
        total_penalty_factor = 1.0 
        if kem_score < KEM_THRESHOLD_FOR_PENALTY:
             if num_tokens > MAX_TOKENS_FOR_PENALTY: 
                 total_penalty_factor *= 0.90
        if IRRELEVANT_TOKENS.intersection(category_tokens):
             total_penalty_factor *= IRRELEVANT_TOKENS_PENALTY_FACTOR
             
        # 2c. FÓRMULA HÍBRIDA FINAL
        base_hybrid_score = (COSINE_WEIGHT * cosine_score) + \
                            (JACCARD_WEIGHT * jaccard_score) + \
                            (KEM_WEIGHT * kem_score) + \
                            (CONCISION_WEIGHT * concision_score)
                            
        hybrid_score = base_hybrid_score * total_penalty_factor 
                       
        candidates_with_scores.append((category_text, hybrid_score, cosine_score, jaccard_score, kem_score, concision_score, num_tokens, total_penalty_factor))

    # --- PASO 2: RE-RANKING HÍBRIDO Y SELECCIÓN FINAL (Numérico) ---
    print("\n--- PASO 2: RE-RANKING HÍBRIDO Y SELECCIÓN FINAL ---")
    
    # Ordenamos TODOS los candidatos por el SCORE HÍBRIDO
    reranked_candidates_by_hybrid = sorted(candidates_with_scores, key=lambda item: item[1], reverse=True)
    
    # Filtramos al TOP N final para el log
    top_candidates_log = reranked_candidates_by_hybrid[:TOP_N_CANDIDATES_FINAL]

    # Log Detallado (Mejora de Logs)
    print(f"   -> Top {len(top_candidates_log)} candidatos finales (Ordenados por Score Híbrido):")
    
    # --- CORRECCIÓN DEL BUCLE ---
    # Iteramos sobre 'top_candidates_log' (la variable correcta)
    for i, (cat, h_score, cos, jac, kem, con, num_t, pen) in enumerate(top_candidates_log):
        log_line = f"      {i+1}. '{cat}' | Híbrido:{h_score:.4f} | Tokens:{num_t} | Concisión:{con:.4f} | Penalidad:{pen:.2f}"
        print(log_line)

    # 3. SELECCIÓN FINAL: Usamos el resultado de nuestra fórmula híbrida
    final_choice = reranked_candidates_by_hybrid[0][0]

    print(f"\n   ✅ Categoría final seleccionada: '{final_choice}' (Basada en Mejor Score Híbrido)")
    
    return final_choice


def generate_multianalysis_report_from_rag(text_consulta, collection, num_results_desired=RAG_NUM_RESULTS_DESIRED, num_to_fetch=RAG_NUM_RESULTS_TO_FETCH, filtro_accion=None):
    """(FASE 3) Busca precedentes, verifica relevancia, procesa en QC y genera informes."""
    
    if collection.count() == 0:
        return "Error: La base de datos RAG está vacía. No se pueden buscar precedentes."
        
    print(f"\n--- PASO 3: ANÁLISIS DE PRECEDENTES (BÚSQUEDA ADAPTATIVA) ---")
    print(f"Buscando hasta {num_to_fetch} precedentes para encontrar {num_results_desired} informes válidos...")

    try:
        query_embedding = generate_embeddings([text_consulta])
        if query_embedding is None:
             return "Error: No se pudo generar el embedding de la consulta."
             
        # --- BÚSQUEDA ADAPTATIVA: Intento 1 (Filtro Estricto) ---
        query_params = {
            "query_embeddings": query_embedding.tolist(),
            "n_results": num_to_fetch, 
            "include": ['metadatas', 'documents', 'distances'] 
        }
        
        filtro_aplicado_original = filtro_accion 
        
        if filtro_accion:
            
            # --- CORRECCIÓN CLAVE: Eliminar el prefijo de materia (ej. "PENAL robo...") ---
            filtro_limpio = filtro_accion
            parts = filtro_accion.split(' ', 1)
            if len(parts) > 1 and parts[0].upper() in ['PENAL', 'CIVIL', 'FAMILIAR', 'MERCANTIL']:
                 filtro_limpio = parts[1].strip()
            # -------------------------------------------------------------------------
            
            query_params["where"] = {"delito_o_accion": filtro_limpio}
            print(f"Aplicando filtro estricto a la búsqueda: 'delito_o_accion' = '{filtro_limpio}'")

        results = collection.query(**query_params)
        
        # --- LÓGICA DE RELAJACIÓN DEL FILTRO ---
        if not results or not results.get('ids') or not results['ids'][0] or len(results['ids'][0]) < num_results_desired:
            num_candidatos_estrictos = len(results['ids'][0]) if results and results.get('ids') else 0
            print(f"   ⚠️ El filtro estricto solo arrojó {num_candidatos_estrictos} candidatos.")
            print(f"   -> Intentando Búsqueda Amplia (sin filtro) para encontrar más precedentes...")
            
            # Intento 2 (Sin Filtro, Solo Semántico)
            query_params.pop("where", None) # Eliminar el filtro
            results = collection.query(**query_params)
            filtro_aplicado_original = None # El filtro ya no aplica

            if not results or not results.get('ids') or not results['ids'][0]:
                 return "No se encontraron precedentes relevantes ni con el filtro estricto ni con la búsqueda amplia."

    except Exception as e:
        print(f"ERROR CRÍTICO durante búsqueda de precedente: {e}")
        return "Error buscando precedentes en la base de datos."

    if not results or not results.get('ids') or not results['ids'][0]:
        return "No se encontraron precedentes relevantes ni con el filtro estricto ni con la búsqueda amplia."

    valid_reports = []
    num_candidates_found = len(results['ids'][0])
    print(f"\nSe encontraron {num_candidates_found} precedentes candidatos para análisis. Analizando en secuencia...")

    for i in range(num_candidates_found):
        # 1. Condición de Parada
        if len(valid_reports) >= num_results_desired:
            break

        print(f"\n--- Intentando analizar Precedente Candidato {i+1}/{num_candidates_found} ---")
        
        # 2. Extracción de Metadatos
        try:
            doc_id = results['ids'][0][i]
            doc_text = results['documents'][0][i]
            metadata = results['metadatas'][0][i]
            distance = results['distances'][0][i]
            materia = metadata.get('materia_principal', 'desconocida').lower()
            accion = metadata.get('delito_o_accion', 'desconocida')
            print(f"Precedente: ID='{doc_id}', Materia='{materia.upper()}', Acción='{accion}', Distancia='{distance:.4f}'")
        except (IndexError, KeyError) as e:
            print(f"   ⚠️ Error al procesar metadatos del Precedente. Descartando.")
            continue

        # --- ¡NUEVO PASO DE VERIFICACIÓN DE RELEVANCIA! ---
        print(f"-> Verificando relevancia de {doc_id}...")
        # Usar solo los primeros 2000 caracteres para una verificación rápida
        fragmento_para_verificar = doc_text[:2000] 
        
        prompt_verif = PROMPT_VERIFICACION_RELEVANCIA.format(
            texto_consulta=text_consulta, 
            fragmento_documento=fragmento_para_verificar
        )
        
        # Usamos la misma función call_local_llm
        veredicto = call_local_llm(prompt_verif)

        # Si el LLM no responde 'si' (o responde 'no', 'falla', etc.), descartamos el documento
        if 'si' not in veredicto.lower():
            print(f"   ⚠️ Precedente {doc_id} descartado. El contenido NO es relevante para la consulta (Veredicto: {veredicto}).")
            continue
        
        print(f"   ✅ Relevancia confirmada. Procediendo a análisis profundo de {doc_id}.")
        # --- FIN DEL NUEVO PASO ---

        # 3. Selección de Prompts
        prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_DEFAULT, PROMPT_REPORTE_DEFAULT
        
        if materia == 'penal': 
            prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_PENAL, PROMPT_REPORTE_PENAL
        elif materia == 'civil':
            prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_CIVIL, PROMPT_REPORTE_CIVIL
        elif materia == 'familiar':
            prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_FAMILIAR, PROMPT_REPORTE_FAMILIAR
        elif materia == 'mercantil':
            prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_MERCANTIL, PROMPT_REPORTE_MERCANTIL
        
        # 4. Extracción de Hechos
        print(f"-> Extrayendo hechos de {doc_id}...")
        extracted_facts = call_local_llm(prompt_extraccion.format(contexto=doc_text))

        # 5. Control de Calidad (Extracción)
        if not extracted_facts or any(keyword in extracted_facts for keyword in FAILURE_KEYWORDS):
            print(f"   ⚠️ Fallo en la extracción de hechos para {doc_id}. Descartando precedente.")
            continue

        # 6. Generación de Informe
        print(f"-> Generando informe para {doc_id}...")
        final_report_body = call_local_llm(prompt_reporte.format(hechos=extracted_facts))
        
        # 7. Control de Calidad (Generación)
        if not final_report_body or any(keyword in final_report_body for keyword in FAILURE_KEYWORDS):
            print(f"   ⚠️ Fallo en la generación del informe para {doc_id}. Descartando precedente.")
            continue

        # 8. Informe Válido (Formato Final) - MEJORADO Y DETALLADO
        report_for_one_doc = f"""
### 📜 Análisis del Precedente {len(valid_reports) + 1}/{num_results_desired}
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `{doc_id}`
* **Materia Principal:** `{materia.upper()}`
* **Acción/Delito Indexado:** `{accion}`
* **Distancia Semántica:** `{distance:.4f}`
* **Fuente del Contenido:** {'Filtro Estricto' if filtro_aplicado_original else 'Búsqueda Amplia (Fallback)'}

---
{final_report_body}
"""
        valid_reports.append(report_for_one_doc)
        print(f"   ✅ Informe válido generado. ({len(valid_reports)}/{num_results_desired} completados)")


    if not valid_reports:
        return "Se encontraron precedentes, pero el LLM no pudo generar un análisis válido para ninguno de ellos (o todos fueron irrelevantes)."

    # 9. Retorno del Informe Final
    return "\n\n---\n\n".join(valid_reports)

In [15]:
# 5. BLOQUE DE EJECUCIÓN (FASE 1)
# ---------------------------------------------------------------------------------------

if __name__ == "__main__":
    
    # --- INICIALIZAR HERRAMIENTAS ---
    if not initialize_spacy_model() or not get_embedding_model_and_tokenizer():
        print("\nDeteniendo ejecución debido a un error en la carga de modelos.")
        exit(1)
        
    # --- EJECUTAR FASE 1 (OPCIONAL: Si los PDFs no están procesados) ---
    # Descomenta las siguientes dos líneas si necesitas (re)procesar los PDFs a Parquet.
    
    # print("--- [EJECUTANDO FASE 1] ---")
    #fase1_procesar_pdfs() 


Cargando modelo de SpaCy (es_core_news_sm)...
✅ Modelo SpaCy cargado.
Cargando modelo de embeddings...
Modelo en dispositivo: cuda


In [ ]:
# 5. BLOQUE DE EJECUCIÓN (FASE 2)
# ---------------------------------------------------------------------------------------

if __name__ == "__main__":
    
    # --- INICIALIZAR HERRAMIENTAS ---
    if not initialize_spacy_model() or not get_embedding_model_and_tokenizer():
        print("\nDeteniendo ejecución debido a un error en la carga de modelos.")
        exit(1)
        

    # --- EJECUTAR FASE 2 (OPCIONAL: Si la DB no está indexada) ---
    # Descomenta las siguientes dos líneas si necesitas (re)indexar ChromaDB desde los Parquet.
    
    # print("--- [EJECUTANDO FASE 2] ---")
    #fase2_indexar_rag()


--- INICIANDO FASE 2: INDEXACIÓN (Versión Simplificada) ---
Inicializando cliente de ChromaDB en la ruta: './chroma_db_juridica_7'
Detectada colección existente 'sentencias_judiciales'. Eliminando para indexación limpia...
DB y colección 'sentencias_judiciales' listas. Documentos actuales: 0.

--- Procesando Lote de Archivos 1/115: batch_0001.parquet ---
Generando embeddings para 499 documentos en 10 sub-lotes de 50...
   -> Sub-lote 1/10 procesado OK.
   -> Sub-lote 2/10 procesado OK.
   -> Sub-lote 3/10 procesado OK.
   -> Sub-lote 4/10 procesado OK.
   -> Sub-lote 5/10 procesado OK.
   -> Sub-lote 6/10 procesado OK.
   -> Sub-lote 7/10 procesado OK.
   -> Sub-lote 8/10 procesado OK.
   -> Sub-lote 9/10 procesado OK.
   -> Sub-lote 10/10 procesado OK.
   -> ÉXITO: 499 documentos de este lote fueron añadidos a ChromaDB.

--- Procesando Lote de Archivos 2/115: batch_0002.parquet ---
Generando embeddings para 500 documentos en 10 sub-lotes de 50...
   -> Sub-lote 1/10 procesado OK.
   

In [25]:
# 3. Lista de Consultas de Ejemplo
lista_consultas_ejemplo = [
    "Se acusa a un individuo de robo calificado por haber sustraído con violencia un teléfono móvil en la vía pública durante la noche. La víctima lo identificó plenamente y existen cámaras de seguridad que registraron el hecho.",
    "Dos personas riñeron en un bar, resultando una de ellas con lesiones que tardan más de 15 días en sanar. Se imputa el delito de lesiones dolosas al agresor principal, quien argumenta legítima defensa.",
    "Un conductor, presuntamente bajo los efectos del alcohol, atropelló a un peatón causándole la muerte. Se le imputa homicidio culposo agravado. La defensa argumenta que el peatón cruzó imprudentemente.",
    "Se denuncia a una persona por fraude, ya que solicitó un anticipo para realizar una obra de construcción y desapareció sin iniciar los trabajos ni devolver el dinero.",
    "Se ejercita acción cambiaria directa por falta de pago de un pagaré vencido. El demandado se niega a pagar alegando que su firma fue falsificada.",
    "Una empresa demanda a otra por incumplimiento de contrato de suministro de mercancías, ya que la entrega se realizó fuera de plazo y con productos defectuosos. Se reclaman daños y perjuicios.",
    "Se demanda el pago de un cheque que fue devuelto por insuficiencia de fondos. El librador argumenta que el cheque fue robado y llenado sin su consentimiento.",
    "Se demanda la rescisión de un contrato de arrendamiento por falta de pago de rentas por más de tres meses. El inquilino argumenta problemas económicos imprevistos debido a la pérdida de su empleo y solicita una prórroga.",
    "El propietario de un terreno demanda la acción reivindicatoria contra el poseedor actual, quien construyó una barda invadiendo parte de su propiedad. El demandado alega prescripción adquisitiva.",
    "Se demanda el pago de daños y perjuicios por un accidente vehicular causado por negligencia. El vehículo del demandante sufrió pérdida total. El demandado niega su responsabilidad.",
    "Un comprador demanda la rescisión de un contrato de compraventa de un inmueble debido a vicios ocultos (humedades estructurales) no informados por el vendedor.",
    "Se promueven diligencias de información ad perpetuam para acreditar la posesión de un inmueble no inscrito en el Registro Público, con el fin de obtener un título de propiedad por prescripción.",
    "Se solicita el divorcio necesario por causal de abandono del hogar conyugal por más de seis meses sin causa justificada. Se pide la disolución del vínculo matrimonial y la liquidación de la sociedad conyugal.",
    "Se demanda el aumento de la pensión alimenticia para dos menores de edad, argumentando un incremento en las necesidades escolares y médicas, y una mejora en la capacidad económica del deudor alimentario.",
    "Un progenitor solicita la modificación de la guarda y custodia de un menor, alegando que el otro progenitor expone al niño a un ambiente inadecuado y negligencia en sus cuidados.",
    "Se demanda la regulación del régimen de convivencias con un menor, ya que el progenitor custodio impide u obstaculiza el contacto del niño con el progenitor no custodio.",
    "Un adolescente es señalado por sustraer mercancía de una tienda departamental. Se busca determinar la medida socioeducativa aplicable conforme a la Ley Nacional del Sistema Integral de Justicia Penal para Adolescentes.",
    "Se promueve amparo indirecto contra una orden de aprehensión por considerarla violatoria de derechos fundamentales, argumentando falta de fundamentación y motivación."
]

In [ ]:
# 5. BLOQUE DE EJECUCIÓN (MAIN)
# ---------------------------------------------------------------------------------------

if __name__ == "__main__":
    
    # --- INICIALIZAR HERRAMIENTAS ---
    if not initialize_spacy_model() or not get_embedding_model_and_tokenizer():
        print("\nDeteniendo ejecución debido a un error en la carga de modelos.")
        exit(1)

    # --- EJECUTAR FASE 3 (Carga y Consultas) ---
    print("\n--- INICIANDO FASE 3: CARGA DE CACHÉ Y EJECUCIÓN DE CONSULTAS ---")

    # 1. Conectar a la Base de Datos
    print("--- CONECTANDO A LA BASE DE DATOS VECTORIAL EXISTENTE ---")
    try:
        client = chromadb.PersistentClient(path=DB_PATH)
        collection = client.get_collection(name="sentencias_judiciales")
        db_count = collection.count()
        if db_count == 0:
            print("ERROR: La colección 'sentencias_judiciales' está vacía. Ejecuta la Fase 2 primero.")
            exit(1)
        print(f"✅ Conexión exitosa. La base de datos contiene {db_count} documentos.")
    except Exception as e:
        print(f"ERROR: No se pudo obtener la colección 'sentencias_judiciales'. Asegúrate de ejecutar la Fase 2 primero. Error: {e}")
        exit(1)

    # 2. Cargar o Generar Caché de Categorías (Lógica de Arranque Rápido)
    if os.path.exists(CACHE_FILE_CATEGORIES) and os.path.exists(CACHE_FILE_EMBEDDINGS):
        print("-> Cargando categorías y embeddings desde el caché local...")
        with open(CACHE_FILE_CATEGORIES, 'r', encoding='utf-8') as f:
            real_categories_raw = json.load(f) 
        _real_categories_embeddings = np.load(CACHE_FILE_EMBEDDINGS)
        
        # CORRECCIÓN DE LIMPIEZA (Para cachés guardados con formato incorrecto)
        real_categories = []
        for item in real_categories_raw:
             if isinstance(item, (list, tuple)):
                 real_categories.append(" ".join(map(str, item))) # Fallback seguro
             else:
                 real_categories.append(str(item))

        print(f"✅ {len(real_categories)} categorías cargadas desde caché.")
    else:
        print("-> No se encontró caché. Generando desde la base de datos (puede tardar)...")
        real_categories = get_real_categories_from_db(collection)
        
        if not real_categories:
            print("\n" + "="*60 + "\n❌ ERROR CRÍTICO: No se encontraron categorías en la base de datos.\n" + "="*60)
            exit(1)

        print("-> Generando embeddings para todas las categorías reales...")
        _real_categories_embeddings = generate_embeddings(real_categories)
        
        if _real_categories_embeddings is not None:
            print("-> Guardando categorías y embeddings en caché...")
            with open(CACHE_FILE_CATEGORIES, 'w', encoding='utf-8') as f:
                json.dump(real_categories, f) 
            np.save(CACHE_FILE_EMBEDDINGS, _real_categories_embeddings)
            print("✅ Caché nuevo guardado.")

    if not real_categories or _real_categories_embeddings is None:
        print("ERROR FATAL: No se pudieron cargar o generar las categorías/embeddings.")
        exit(1)


    # --- 4. Ejecución de las Consultas ---
    for i, consulta_usuario in enumerate(lista_consultas_ejemplo):
        print(f"\n\n{'='*25}\n--- NUEVA CONSULTA ({i+1}/{len(lista_consultas_ejemplo)}) ---\n{'='*25}")
        print(f"> {consulta_usuario}\n")
        
        start_time_consulta = time.time()
        
        # 2. Clasificación Híbrida (Selección del filtro)
        categoria_final_para_filtro = classify_dynamically(
            consulta_usuario,
            real_categories,
            _real_categories_embeddings
        )

        # 3. Generación del Informe RAG (Usando el filtro seleccionado y la búsqueda adaptativa)
        informe_final = generate_multianalysis_report_from_rag(
            text_consulta=consulta_usuario,
            collection=collection,
            filtro_accion=categoria_final_para_filtro
        )
        
        end_time_consulta = time.time()
        
        # 4. Presentación de Resultados
        output_markdown = f"""
# Resultados del Co-piloto Jurídico
**Consulta Analizada:** > {consulta_usuario}
**Categoría Final para Filtro:** `{categoria_final_para_filtro or 'N/A (Fallo en clasificación)'}`
**Tiempo de Procesamiento:** `{end_time_consulta - start_time_consulta:.2f} segundos`
***
## 📚 Análisis Detallado de Precedentes
{informe_final}
"""
        print("\n\n" + "="*25 + " INFORME FINAL " + "="*25)
        print(output_markdown)
        
        try:
            display(Markdown(output_markdown))
        except NameError:
            pass
            
        time.sleep(2) # Pausa breve entre consultas


--- INICIANDO FASE 3: CARGA DE CACHÉ Y EJECUCIÓN DE CONSULTAS ---
--- CONECTANDO A LA BASE DE DATOS VECTORIAL EXISTENTE ---
✅ Conexión exitosa. La base de datos contiene 55584 documentos.
-> Cargando categorías y embeddings desde el caché local...
✅ 1053 categorías cargadas desde caché.


--- NUEVA CONSULTA (1/18) ---
> Se acusa a un individuo de robo calificado por haber sustraído con violencia un teléfono móvil en la vía pública durante la noche. La víctima lo identificó plenamente y existen cámaras de seguridad que registraron el hecho.


--- PASO 1: BÚSQUEDA SEMÁNTICA INICIAL (FILTRO GRUESO) ---

--- PASO 2a: EXTRACCIÓN DE ENTIDADES CLAVE ---
   -> Entidades extraídas: 'robo calificado, violencia, identificación de la víctima'
   -> KEM Tokens Dinámicos: ['violencia', 'robo', 'calificado', 'víctima', 'identificación']

--- PASO 2: RE-RANKING HÍBRIDO Y SELECCIÓN FINAL ---
   -> Top 10 candidatos finales (Ordenados por Score Híbrido):
      1. 'PENAL robo-calificado-y-privacion-ileg


# Resultados del Co-piloto Jurídico
**Consulta Analizada:** > Se acusa a un individuo de robo calificado por haber sustraído con violencia un teléfono móvil en la vía pública durante la noche. La víctima lo identificó plenamente y existen cámaras de seguridad que registraron el hecho.
**Categoría Final para Filtro:** `PENAL robo-calificado-y-privacion-ilegal-de-la-libertad`
**Tiempo de Procesamiento:** `195.08 segundos`
***
## 📚 Análisis Detallado de Precedentes

### 📜 Análisis del Precedente 1/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2023-06-29-134149_penal_robo-calificado-y-privacion-ilegal-de-la-libertad.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `robo-calificado-y-privacion-ilegal-de-la-libertad`
* **Distancia Semántica:** `5.5663`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis penal
### 1. resumen ejecutivo
el imputado fue vinculado a proceso por la comisión de un delito doloso, específicamente por privación ilegítima de libertad que generó riesgos para la víctima. el tribunal decidió otorgar la prisión preventiva justificada debido a la gravedad del delito y los riesgos identificados. la decisión fue confirmada sin necesidad de respuesta al recurso de apelación adhesiva.

### 2. datos de identificación del proceso
- tribunal: [no se proporciona el nombre específico del tribunal en los hechos extraídos, por lo que asumiremos un tribunal penal común]
- expediente: no especificado
- etapa procesal: vinculación a proceso
- delito(s): privación ilegítima de libertad dolosa

### 3. partes involucradas
- acusado(s): el imputado (no se proporciona el nombre)
- víctima(s)/ofendido(s): la víctima que sufrió privación ilegítima de libertad

### 4. análisis de hechos y decisión
hechos clave:
el imputado fue vinculado a proceso por la comisión de un delito doloso, específicamente por privación ilegítima de libertad que generó riesgos para la víctima. la víctima sufrió una situación de peligro y vulnerabilidad debido a esta acción.

decisión del tribunal:
el tribunal decidió otorgar la prisión preventiva justificada, lo que indica que el imputado permanecerá en custodia mientras se desarrolla el proceso penal para garantizar su comparecencia y evitar riesgos para la víctima o la sociedad.

razonamiento del tribunal (análisis):
la decisión de otorgar la prisión preventiva justificada se basó en la gravedad del delito doloso cometido por el imputado, específicamente la privación ilegítima de libertad que generó riesgos para la víctima. el tribunal consideró que esta medida era necesaria para garantizar la seguridad y protección de la víctima, así como para asegurar la comparecencia del imputado en el proceso penal.

la decisión fue proporcional, adecuada y respetuosa de derechos humanos, ya que se tomó en consideración la situación específica de la víctima y las necesidades de seguridad pública. el tribunal también demostró un compromiso con los principios del debido proceso y el derecho a la defensa, al otorgar la oportunidad para que la defensa presentara sus argumentos.

en resumen, la decisión del tribunal se basó en una evaluación cuidadosa de las circunstancias del caso, considerando tanto la gravedad del delito como las necesidades de seguridad y protección de la víctima. la medida adoptada fue proporcionada a los hechos y respetuosa con los derechos humanos y el debido proceso.


---


### 📜 Análisis del Precedente 2/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2022-04-22-012548_penal_robo-calificado-y-privacion-ilegal-de-la-libertad.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `robo-calificado-y-privacion-ilegal-de-la-libertad`
* **Distancia Semántica:** `5.5816`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis penal
### 1. resumen ejecutivo
el tribunal dictaminó la vinculación a proceso de varios imputados por delitos como robo calificado, privación ilegal de libertad y portación de arma de fuego. se decreta también la prisión preventiva para uno de los imputados durante dos meses.

### 2. datos de identificación del proceso
- tribunal: tribunal de justicia del estado de jalisco.
- expediente: no se proporciona el número de expediente en los hechos extraídos.
- etapa procesal: vinculación a proceso y medida cautelar.
- delito(s): robo calificado, privación ilegal de libertad y portación de arma de fuego.

### 3. partes involucradas
- acusado(s): varios imputados.
- víctima(s)/ofendido(s): no se menciona a la víctima en los hechos extraídos.

### 4. análisis de hechos y decisión

hechos clave:
el tribunal dictó auto de vinculación a proceso contra varios imputados por su probable participación en delitos como robo calificado, privación ilegal de libertad y portación de arma de fuego. se decreta la prisión preventiva para uno de los imputados durante dos meses.

decisión del tribunal:
la decisión se basa en los artículos 20 fracción i, 67, 68, 70, 133 fracción iii, 461, 467 fracción v, 475 y 479 del código nacional de procedimientos penales.

razonamiento del tribunal (análisis):
la decisión se basa en la aplicación de los artículos mencionados. el tribunal consideró que el imputado fue vinculado a proceso en términos del numeral 233 en relación al 236 fracción vii y ix del código penal del estado, lo cual quedó acreditado. la prisión preventiva se decreta para uno de los imputados durante dos meses a efecto de que se lleve a cabo el cierre de investigación y se continúe con la secuela legal respecto de la medida cautelar impuesta.

la aplicación de los artículos 20 fracción i, 67, 68, 70, 133 fracción iii, 461, 467 fracción v, 475 y 479 del código nacional de procedimientos penales se basa en la necesidad de asegurar el proceso penal y garantizar que se lleve a cabo una investigación exhaustiva. la prisión preventiva se decreta para uno de los imputados debido a la gravedad de los delitos cometidos y la necesidad de asegurar su comparecencia ante el tribunal.

en resumen, la decisión del tribunal se basa en la aplicación de las leyes penales y procesales para garantizar que se lleve a cabo una investigación exhaustiva y se asegure la comparecencia de los imputados.


---


### 📜 Análisis del Precedente 3/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2022-04-13-113623_penal_robo-calificado-y-privacion-ilegal-de-la-libertad.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `robo-calificado-y-privacion-ilegal-de-la-libertad`
* **Distancia Semántica:** `6.2644`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis penal

### 1. resumen ejecutivo
el caso involucra a [no.159]_eliminado_nombre_del_imputado_[97] por su probable responsabilidad en el delito de robo calificado, cometido en agravio de la víctima [no.160]_eliminado_el_nombre_completo_[1]. el tribunal decreta vinculación a proceso al imputado por este delito y auto de no vinculación a proceso por el delito de privación ilegal de la libertad, ya que no se ha acreditado el hecho.

### 2. datos de identificación del proceso
- tribunal: juez octavo de control, enjuiciamiento y justicia integral para adolescentes del primer distrito judicial en el estado de jalisco.
- expediente: carpeta procesal 5444/2021.
- etapa procesal: investigación y decisión sobre medidas cautelares.
- delito(s): robo calificado y privación ilegal de la libertad.

### 3. partes involucradas
- acusado(s): [no.159]_eliminado_nombre_del_imputado_[97].
- víctima(s)/ofendido(s): [no.160]_eliminado_el_nombre_completo_[1] y [no.161]_eliminado_el_nombre_completo_[1].

### 4. análisis de hechos y decisión

hechos clave:
se investiga a [no.159]_eliminado_nombre_del_imputado_[97] por su probable responsabilidad en el delito de robo calificado cometido contra la víctima [no.160]_eliminado_el_nombre_completo_[1], y también se le involucra al ofendido [no.161]_eliminado_el_nombre_completo_[1]. además, se investiga a [no.162]_eliminado_nombre_del_imputado_[97] por el delito de privación ilegal de la libertad en agravio de [no.163]_eliminado_el_nombre_completo_[1], pero no se ha acreditado este hecho.

decisión del tribunal:
se decreta vinculación a proceso a [no.159]_eliminado_nombre_del_imputado_[97] por su probable responsabilidad en el delito de robo calificado y auto de no vinculación a proceso a favor de [no.162]_eliminado_nombre_del_imputado_[97] por el delito de privación ilegal de la libertad.

razonamiento del tribunal (análisis):
la decisión del tribunal se basa en la existencia de pruebas suficientes para vincular a [no.159]_eliminado_nombre_del_imputado_[97] al delito de robo calificado, considerando el testimonio y las evidencias presentadas en el proceso. por otro lado, no se han encontrado pruebas que respalden la acusación por privación ilegal de la libertad contra [no.162]_eliminado_nombre_del_imputado_[97], lo que lleva a la decisión de auto de no vinculación a proceso.

la aplicación del código procesal penal nacional se enfoca en garantizar el debido proceso y la protección de los derechos de las partes involucradas. la instrucción al juez para señalar fecha para celebrar la audiencia prevista por el numeral 161 del código procesal penal nacional refleja la importancia de avanzar en el proceso judicial con transparencia y celeridad.

en resumen, la decisión del tribunal se basa en la evaluación cuidadosa de las pruebas presentadas y la aplicación estricta de los principios legales que rigen el proceso penal.





--- NUEVA CONSULTA (2/18) ---
> Dos personas riñeron en un bar, resultando una de ellas con lesiones que tardan más de 15 días en sanar. Se imputa el delito de lesiones dolosas al agresor principal, quien argumenta legítima defensa.


--- PASO 1: BÚSQUEDA SEMÁNTICA INICIAL (FILTRO GRUESO) ---

--- PASO 2a: EXTRACCIÓN DE ENTIDADES CLAVE ---
   -> Entidades extraídas: 'lesiones dolosas, legítima defensa, responsabilidad penal'
   -> KEM Tokens Dinámicos: ['defensa', 'dolosa', 'responsabilidad', 'penal', 'legítimo', 'lesión']

--- PASO 2: RE-RANKING HÍBRIDO Y SELECCIÓN FINAL ---
   -> Top 10 candidatos finales (Ordenados por Score Híbrido):
      1. 'PENAL lesiones-dolosas-y-delitos-cometidos-contra-representantes-de-la-autoridad' | Híbrido:0.3865 | Tokens:7 | Concisión:0.3780 | Penalidad:0.90
      2. 'PENAL amenazas-contra-la-dignidad-de-las-personas-y-lesiones' | Híbrido:0.3569 | Tokens:5 | Concisión:0.4472 | Penalidad:1.00
      3. 'PENAL lesiones-a-titulo-de-culpa-por-imprudencia' 

KeyboardInterrupt: 

In [12]:
def generate_final_synthesis(consulta, informes_validos):
    """
    Toma la lista de informes generados y crea una síntesis final usando el LLM.
    """
    if not informes_validos:
        return "Error: No se pudieron generar informes válidos para la síntesis."

    # Prepara las entradas para el prompt, manejando casos de menos de 3 informes
    informe_1 = informes_validos[0] if len(informes_validos) > 0 else "N/A"
    informe_2 = informes_validos[1] if len(informes_validos) > 1 else "N/A"
    informe_3 = informes_validos[2] if len(informes_validos) > 2 else "N/A"

    prompt_sintesis = PROMPT_SINTESIS_FINAL.format(
        texto_consulta=consulta,
        informe_1=informe_1,
        informe_2=informe_2,
        informe_3=informe_3
    )
    
    print("-> Generando síntesis final de los 3 informes...")
    sintesis = call_local_llm(prompt_sintesis)
    
    if not sintesis or any(keyword in sintesis for keyword in FAILURE_KEYWORDS):
        print("   ⚠️ Fallo en la generación de la síntesis. Se devolverán los informes individuales.")
        return "El LLM no pudo generar una síntesis de los informes."
    
    print("   ✅ Síntesis generada exitosamente.")
    return sintesis


In [47]:
def generate_multianalysis_report_from_rag(text_consulta, collection, num_results_desired=RAG_NUM_RESULTS_DESIRED, num_to_fetch=RAG_NUM_RESULTS_TO_FETCH, filtro_accion=None):
    """
    (FASE 3) Busca precedentes, elimina el QC de Relevancia (QC_1) y se enfoca en la Extracción (QC_2/Fallback).
    """
    
    if collection.count() == 0:
        return "Error: La base de datos RAG está vacía. No se pueden buscar precedentes."
        
    print(f"\n--- PASO 3: ANÁLISIS DE PRECEDENTES (BÚSQUEDA ADAPTATIVA) ---")
    print(f"Buscando hasta {num_to_fetch} precedentes para encontrar {num_results_desired} informes válidos...")

    try:
        query_embedding = generate_embeddings([text_consulta])
        if query_embedding is None:
             return "Error: No se pudo generar el embedding de la consulta."
             
        # --- BÚSQUEDA ADAPTATIVA: Intento 1 (Filtro Estricto) ---
        query_params = {
            "query_embeddings": query_embedding.tolist(),
            "n_results": num_to_fetch, 
            "include": ['metadatas', 'documents', 'distances'] 
        }
        
        filtro_aplicado_original = filtro_accion 
        
        if filtro_accion:
            filtro_limpio = filtro_accion
            parts = filtro_accion.split(' ', 1)
            if len(parts) > 1 and parts[0].upper() in ['PENAL', 'CIVIL', 'FAMILIAR', 'MERCANTIL']:
                 filtro_limpio = parts[1].strip()
            
            query_params["where"] = {"delito_o_accion": filtro_limpio}
            print(f"Aplicando filtro estricto a la búsqueda: 'delito_o_accion' = '{filtro_limpio}'")

        results = collection.query(**query_params)
        
        # --- LÓGICA DE RELAJACIÓN DEL FILTRO ---
        if not results or not results.get('ids') or not results['ids'][0] or len(results['ids'][0]) < num_results_desired:
            num_candidatos_estrictos = len(results['ids'][0]) if results and results.get('ids') else 0
            print(f"   ⚠️ El filtro estricto solo arrojó {num_candidatos_estrictos} candidatos.")
            print(f"   -> Intentando Búsqueda Amplia (sin filtro) para encontrar más precedentes...")
            
            query_params.pop("where", None) # Eliminar el filtro
            results = collection.query(**query_params)
            filtro_aplicado_original = None # El filtro ya no aplica

            if not results or not results.get('ids') or not results['ids'][0]:
                 return "No se encontraron precedentes relevantes ni con el filtro estricto ni con la búsqueda amplia."

    except Exception as e:
        print(f"ERROR CRÍTICO durante búsqueda de precedente: {e}")
        return "Error buscando precedentes en la base de datos."

    if not results or not results.get('ids') or not results['ids'][0]:
        return "No se encontraron precedentes relevantes ni con el filtro estricto ni con la búsqueda amplia."

    # --- INICIO DEL BUCLE DE ANÁLISIS ---
    valid_reports = []
    num_candidates_found = len(results['ids'][0])
    print(f"\nSe encontraron {num_candidates_found} precedentes candidatos para análisis. Analizando en secuencia...")

    for i in range(num_candidates_found):
        if len(valid_reports) >= num_results_desired:
            break

        print(f"\n--- Intentando analizar Precedente Candidato {i+1}/{num_candidates_found} ---")
        
        try:
            doc_id = results['ids'][0][i]
            doc_text = results['documents'][0][i]
            metadata = results['metadatas'][0][i]
            distance = results['distances'][0][i]
            materia = metadata.get('materia_principal', 'desconocida').lower()
            accion = metadata.get('delito_o_accion', 'desconocida')
            print(f"Precedente: ID='{doc_id}', Materia='{materia.upper()}', Acción='{accion}', Distancia='{distance:.4f}'")
        except (IndexError, KeyError) as e:
            print(f"   ⚠️ Error al procesar metadatos del Precedente. Descartando.")
            continue

        # --- SE ELIMINA QC_1: Ya no verificamos la relevancia, confiamos en la distancia semántica ---
        
        # 3. Selección de Prompts Específicos (Primer Intento)
        prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_DEFAULT, PROMPT_REPORTE_DEFAULT
        
        if materia == 'penal': 
            prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_PENAL, PROMPT_REPORTE_PENAL
        elif materia == 'civil':
            prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_CIVIL, PROMPT_REPORTE_CIVIL
        elif materia == 'familiar':
            prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_FAMILIAR, PROMPT_REPORTE_FAMILIAR
        elif materia == 'mercantil':
            prompt_extraccion, prompt_reporte = PROMPT_EXTRACCION_MERCANTIL, PROMPT_REPORTE_MERCANTIL
        
        # 4. Extracción de Hechos (Primer Intento: Específico)
        print(f"-> Extrayendo hechos de {doc_id} con prompt específico ({materia.upper()})...")
        extracted_facts = call_local_llm(prompt_extraccion.format(contexto=doc_text))

        # --- INICIO DE LA NUEVA LÓGICA DE FALLBACK DE EXTRACCIÓN (DOBLE EXTRACCIÓN) ---
        
        if not extracted_facts or any(keyword in extracted_facts for keyword in FAILURE_KEYWORDS):
            # Fallo en el Prompt ESPECÍFICO (QC_2)
            print(f"   ⚠️ Fallo en la extracción específica. Intentando Extracción Genérica de Fallback...")
            
            # Segundo Intento: Extracción Genérica
            prompt_extraccion_fallback = PROMPT_EXTRACCION_DEFAULT 
            prompt_reporte_fallback = PROMPT_REPORTE_DEFAULT
            extracted_facts = call_local_llm(prompt_extraccion_fallback.format(contexto=doc_text))
            
            # Verificación final del fallback (QC_2b)
            if not extracted_facts or any(keyword in extracted_facts for keyword in FAILURE_KEYWORDS):
                print(f"   ❌ Fallo en la extracción genérica también. Descartando precedente.")
                continue
            
            print("   ✅ Extracción genérica exitosa.")
            prompt_reporte = prompt_reporte_fallback # Si funcionó, usamos el reporte genérico
        # --- FIN DE LA NUEVA LÓGICA DE FALLBACK DE EXTRACCIÓN ---

        # 5. Generación de Informe
        print(f"-> Generando informe para {doc_id}...")
        final_report_body = call_local_llm(prompt_reporte.format(hechos=extracted_facts))
        
        # 6. Control de Calidad 2 (QC_2): Verificación de Alucinación/Falla en el Reporte
        if not final_report_body or any(keyword in final_report_body for keyword in FAILURE_KEYWORDS):
            print(f"   ⚠️ Fallo en la generación del informe para {doc_id}. Descartando precedente.")
            continue

        # 7. Informe Válido (Formato Final) - MEJORADO Y DETALLADO
        report_for_one_doc = f"""
### 📜 Análisis del Precedente {len(valid_reports) + 1}/{num_results_desired}
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `{doc_id}`
* **Materia Principal:** `{materia.upper()}`
* **Acción/Delito Indexado:** `{accion}`
* **Distancia Semántica:** `{distance:.4f}`
* **Fuente del Contenido:** {'Filtro Estricto' if filtro_aplicado_original else 'Búsqueda Amplia (Fallback)'}

---
{final_report_body}
"""
        valid_reports.append(report_for_one_doc)
        print(f"   ✅ Informe válido generado. ({len(valid_reports)}/{num_results_desired} completados)")


    if not valid_reports:
        return "No se encontraron informes válidos para sintetizar."

    # --- PASO 9: SÍNTESIS FINAL (Si se encontró 1 o más) ---
    
    informes_individuales_texto = "\n\n---\n\n".join(valid_reports)

    if len(valid_reports) < 2:
        print(f"\n--- PASO 4: SÍNTESIS OMITIDA (Solo se encontró {len(valid_reports)} informe válido) ---")
        
        informe_final_completo = f"""
## 💎 Síntesis de Precedentes Relevantes
No se encontraron suficientes precedentes (se requieren 2) para realizar un análisis comparativo. 
Se presenta el único informe válido encontrado:
***
{informes_individuales_texto}
"""
        return informe_final_completo

    # Si encontramos 2 o más, SÍ intentamos la síntesis
    print(f"\n--- PASO 4: SINTETIZANDO {len(valid_reports)} INFORMES ---")
    
    global PROMPT_SINTESIS_FINAL 
    
    prompt_sintesis = PROMPT_SINTESIS_FINAL.format(
        text_consulta=text_consulta,
        informes_precedentes=informes_individuales_texto # Se pasa el bloque completo de informes
    )
        
    sintesis_final = call_local_llm(prompt_sintesis)
    
    if not sintesis_final or any(keyword in sintesis_final for keyword in FAILURE_KEYWORDS):
        print("   ⚠️ Fallo en la generación de la Síntesis Final. Devolviendo informes individuales.")
        return f"""
## 💎 Síntesis de Precedentes Relevantes
(El LLM no pudo generar la síntesis comparativa. Se presentan los informes individuales.)
***
{informes_individuales_texto}
"""
    else:
        print("   ✅ Síntesis generada exitosamente.")
        informe_final_completo = f"""
{sintesis_final}
***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales
{informes_individuales_texto}
"""
        return informe_final_completo

In [50]:
# 5. BLOQUE DE EJECUCIÓN (MAIN CON SÍNTESIS FINAL)
# ---------------------------------------------------------------------------------------

if __name__ == "__main__":
    
    # --- INICIALIZAR HERRAMIENTAS ---
    if not initialize_spacy_model() or not get_embedding_model_and_tokenizer():
        print("\nDeteniendo ejecución debido a un error en la carga de modelos.")
        # exit(1) # Descomentar en un script .py
    
    # --- EJECUTAR FASE 3 (Carga y Consultas) ---
    print("\n--- INICIANDO FASE 3: CARGA DE CACHÉ Y EJECUCIÓN DE CONSULTAS ---")

    # 1. Conectar a la Base de Datos
    print("--- CONECTANDO A LA BASE DE DATOS VECTORIAL EXISTENTE ---")
    try:
        client = chromadb.PersistentClient(path=DB_PATH)
        collection = client.get_collection(name="sentencias_judiciales")
        db_count = collection.count()
        if db_count == 0:
            print("ERROR: La colección 'sentencias_judiciales' está vacía. Ejecuta la Fase 2 primero.")
            # exit(1) # Descomentar en un script .py
        else:
            print(f"✅ Conexión exitosa. La base de datos contiene {db_count} documentos.")
    except Exception as e:
        print(f"ERROR: No se pudo obtener la colección 'sentencias_judiciales'. Asegúrate de ejecutar la Fase 2 primero. Error: {e}")
        # exit(1) # Descomentar en un script .py

    # 2. Cargar o Generar Caché de Categorías
    if os.path.exists(CACHE_FILE_CATEGORIES) and os.path.exists(CACHE_FILE_EMBEDDINGS):
        print("-> Cargando categorías y embeddings desde el caché local...")
        with open(CACHE_FILE_CATEGORIES, 'r', encoding='utf-8') as f:
            real_categories_raw = json.load(f) 
        _real_categories_embeddings = np.load(CACHE_FILE_EMBEDDINGS)
        
        real_categories = []
        for item in real_categories_raw:
             if isinstance(item, (list, tuple)):
                 real_categories.append(" ".join(map(str, item))) 
             else:
                 real_categories.append(str(item))

        print(f"✅ {len(real_categories)} categorías cargadas desde caché.")
    else:
        print("-> No se encontró caché. Generando desde la base de datos (puede tardar)...")
        real_categories = get_real_categories_from_db(collection)
        
        if not real_categories:
            print("\n" + "="*60 + "\n❌ ERROR CRÍTICO: No se encontraron categorías en la base de datos.\n" + "="*60)
            # exit(1) # Descomentar en un script .py
        else:
            print("-> Generando embeddings para todas las categorías reales...")
            _real_categories_embeddings = generate_embeddings(real_categories)
            
            if _real_categories_embeddings is not None:
                print("-> Guardando categorías y embeddings en caché...")
                with open(CACHE_FILE_CATEGORIES, 'w', encoding='utf-8') as f:
                    json.dump(real_categories, f) 
                np.save(CACHE_FILE_EMBEDDINGS, _real_categories_embeddings)
                print("✅ Caché nuevo guardado.")

    if not 'real_categories' in locals() or _real_categories_embeddings is None:
        print("ERROR FATAL: No se pudieron cargar o generar las categorías/embeddings.")
        # exit(1) # Descomentar en un script .py
    else:
        # --- 4. Ejecución de las Consultas ---
        for i, consulta_usuario in enumerate(lista_consultas_ejemplo):
            print(f"\n\n{'='*25}\n--- NUEVA CONSULTA ({i+1}/{len(lista_consultas_ejemplo)}) ---\n{'='*25}")
            print(f"> {consulta_usuario}\n")
            
            start_time_consulta = time.time()
            
            # 2. Clasificación Híbrida (Selección del filtro)
            categoria_final_para_filtro = classify_dynamically(
                consulta_usuario,
                real_categories,
                _real_categories_embeddings
            )

            # 3. Generación de Informes Individuales (¡AHORA DEVUELVE UNA LISTA!)
            lista_informes_generados = generate_multianalysis_report_from_rag(
                text_consulta=consulta_usuario,
                collection=collection,
                filtro_accion=categoria_final_para_filtro
            )
            
            # 4. (NUEVO) Generar Síntesis Final
            informe_final_sintetizado = ""
            informes_individuales_md = ""
            
            if isinstance(lista_informes_generados, str):
                # Es un mensaje de error (ej. "No se encontraron precedentes")
                informe_final_sintetizado = lista_informes_generados
            elif lista_informes_generados:
                # Es una lista de informes, ¡generar la síntesis!
                print(f"\n--- PASO 4: SINTETIZANDO {len(lista_informes_generados)} INFORMES ---")
                informe_final_sintetizado = generate_final_synthesis(consulta_usuario, lista_informes_generados)
                
                # Formatear los informes individuales para el 'apéndice'
                informes_individuales_md = "\n\n---\n\n".join(lista_informes_generados)
            else:
                informe_final_sintetizado = "No se encontraron informes válidos para sintetizar."

            end_time_consulta = time.time()
            
            # 5. Presentación de Resultados (Formato Unificado)
            output_markdown = f"""
# Resultados del Co-piloto Jurídico
**Consulta Analizada:** > {consulta_usuario}
**Categoría Final para Filtro:** `{categoria_final_para_filtro or 'N/A'}`
**Tiempo de Procesamiento:** `{end_time_consulta - start_time_consulta:.2f} segundos`
***
{informe_final_sintetizado}
***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales
{informes_individuales_md}
"""
            print("\n\n" + "="*25 + " INFORME FINAL " + "="*25)
            print(output_markdown)
            
            try:
                display(Markdown(output_markdown))
            except NameError:
                pass
                
            time.sleep(2) # Pausa breve entre consultas

Cargando modelo de SpaCy (es_core_news_sm)...
✅ Modelo SpaCy cargado.
Cargando modelo de embeddings...
Modelo en dispositivo: cuda

--- INICIANDO FASE 3: CARGA DE CACHÉ Y EJECUCIÓN DE CONSULTAS ---
--- CONECTANDO A LA BASE DE DATOS VECTORIAL EXISTENTE ---
✅ Conexión exitosa. La base de datos contiene 55584 documentos.
-> Cargando categorías y embeddings desde el caché local...
✅ 1053 categorías cargadas desde caché.


--- NUEVA CONSULTA (1/18) ---
> Se acusa a un individuo de robo calificado por haber sustraído con violencia un teléfono móvil en la vía pública durante la noche. La víctima lo identificó plenamente y existen cámaras de seguridad que registraron el hecho.


--- PASO 1: BÚSQUEDA SEMÁNTICA INICIAL (FILTRO GRUESO) ---

--- PASO 2a: EXTRACCIÓN DE ENTIDADES CLAVE ---
   -> Entidades extraídas: 'robo calificado, violencia, identificación de la víctima'
   -> KEM Tokens Dinámicos: ['víctima', 'calificado', 'identificación', 'robo', 'violencia']

--- PASO 2: RE-RANKING HÍBRIDO Y 


# Resultados del Co-piloto Jurídico
**Consulta Analizada:** > Se acusa a un individuo de robo calificado por haber sustraído con violencia un teléfono móvil en la vía pública durante la noche. La víctima lo identificó plenamente y existen cámaras de seguridad que registraron el hecho.
**Categoría Final para Filtro:** `PENAL robo-calificado-y-privacion-ilegal-de-la-libertad`
**Tiempo de Procesamiento:** `152.75 segundos`
***

## síntesis de precedentes relevantes

### 1. resumen ejecutivo (síntesis)
los tres informes analizados se refieren a casos relacionados con delitos penales, específicamente robo calificado y privación ilegal de la libertad. los informes proporcionan detalles sobre los procedimientos legales, las partes involucradas y las decisiones tomadas por el tribunal en cada caso. sin embargo, no se menciona una decisión final explícita en ninguno de los informes.

### 2. precedente más relevante
el primer informe (`2023-02-13-145421_penal_robo-calificado-y-privacion-ilegal-de-la-libertad.pdf`) es el más relevante, ya que proporciona detalles específicos sobre un caso de robo calificado y privación ilegal de la libertad, incluyendo la identificación del tribunal, las partes involucradas y los fundamentos legales aplicados.

### 3. análisis comparativo
 puntos en común: todos los informes se refieren a casos relacionados con delitos penales, específicamente robo calificado y privación ilegal de la libertad.
 diferencias y matices: cada informe tiene detalles únicos sobre el caso analizado. el primer informe proporciona una evaluación detallada de los hechos y pruebas presentadas, mientras que el segundo informe se centra en un juicio civil ordinario sin delitos penales involucrados.

### 4. conclusión para el usuario
basado en la evidencia combinada de los informes, es importante considerar cuidadosamente las pruebas y fundamentos legales aplicados en cada caso al tomar decisiones o emitir juicios sobre casos similares.
***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales

### 📜 Análisis del Precedente 1/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2022-04-22-012548_penal_robo-calificado-y-privacion-ilegal-de-la-libertad.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `robo-calificado-y-privacion-ilegal-de-la-libertad`
* **Distancia Semántica:** `5.5816`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis penal
### 1. resumen ejecutivo
el tribunal dictaminó la vinculación a proceso de varios imputados por delitos como robo calificado, privación ilegal de libertad y portación de arma de fuego. se decreta la prisión preventiva para uno de los imputados durante dos meses debido a la necesidad de cierre de investigación y continuación con la secuela legal. además, se menciona un error involuntario en la audiencia constitucional.

### 2. datos de identificación del proceso
- tribunal: tribunal de justicia del estado de jalisco.
- expediente: no especificado en los hechos extraídos.
- etapa procesal: vinculación a proceso y medida cautelar.
- delito(s): robo calificado, privación ilegal de libertad y portación de arma de fuego.

### 3. partes involucradas
- acusado(s): varios imputados no especificados en los hechos extraídos.
- víctima(s)/ofendido(s): no se mencionan víctimas o ofendidos en los hechos extraídos.

### 4. análisis de hechos y decisión
hechos clave:
el tribunal dictó auto de vinculación a proceso contra varios imputados por su probable participación en delitos como robo calificado, privación ilegal de libertad y portación de arma de fuego. se decreta la prisión preventiva para uno de los imputados durante dos meses a efecto de que se lleve a cabo el cierre de investigación y se continúe con la secuela legal respecto de la medida cautelar impuesta.

decisión del tribunal:
la decisión del tribunal es vincular a proceso a varios imputados por los mencionados delitos y aplicar una prisión preventiva para uno de ellos durante dos meses.

razonamiento del tribunal (análisis):
el tribunal basó su decisión en las leyes aplicables del código nacional de procedimientos penales, específicamente en los artículos 20 fracción i, 67, 68, 70, 133 fracción iii, 461, 467 fracción v, 475 y 479. estos artículos se refieren a la vinculación a proceso, medidas cautelares, procedimiento penal y derechos del imputado. el tribunal consideró que la participación de los imputados en los delitos mencionados era probable y que la prisión preventiva para uno de ellos era necesaria para garantizar el cierre de investigación y continuar con la secuela legal.

la decisión también se basa en el error involuntario en la audiencia constitucional, donde se señaló que el imputado estaba vinculado a proceso en términos del numeral 233 en relación al 236 fracción vii y ix del código penal del estado. sin embargo, atendiendo a lo razonado en la presente resolución, quedan acreditadas las calificativas enumeradas con antelación.

en conclusión, el tribunal tomó su decisión basándose en pruebas y leyes aplicables, garantizando que se siga el debido proceso y se protejan los derechos del imputado.


---


### 📜 Análisis del Precedente 2/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2022-04-13-113623_penal_robo-calificado-y-privacion-ilegal-de-la-libertad.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `robo-calificado-y-privacion-ilegal-de-la-libertad`
* **Distancia Semántica:** `6.2644`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis penal

### 1. resumen ejecutivo
el caso involucra a [no.159]_eliminado_nombre_del_imputado_[97] por su probable responsabilidad en el delito de robo calificado, cometido en agravio de la víctima [no.160]_eliminado_el_nombre_completo_[1]. el tribunal decreta vinculación a proceso al imputado por este delito y auto de no vinculación a proceso por el delito de privación ilegal de la libertad, ya que no se ha acreditado el hecho.

### 2. datos de identificación del proceso
- tribunal: juez octavo de control, enjuiciamiento y justicia integral para adolescentes del primer distrito judicial en el estado de jalisco.
- expediente: carpeta procesal 5444/2021.
- etapa procesal: investigación y decisión sobre medidas cautelares.
- delito(s): robo calificado y privación ilegal de la libertad.

### 3. partes involucradas
- acusado(s): [no.159]_eliminado_nombre_del_imputado_[97].
- víctima(s)/ofendido(s): [no.160]_eliminado_el_nombre_completo_[1] y [no.161]_eliminado_el_nombre_completo_[1].

### 4. análisis de hechos y decisión

hechos clave:
se investiga a [no.159]_eliminado_nombre_del_imputado_[97] por su probable responsabilidad en el delito de robo calificado cometido contra la víctima [no.160]_eliminado_el_nombre_completo_[1], y también se le involucra al ofendido [no.161]_eliminado_el_nombre_completo_[1]. además, se investiga a [no.162]_eliminado_nombre_del_imputado_[97] por el delito de privación ilegal de la libertad en agravio de [no.163]_eliminado_el_nombre_completo_[1], pero no se ha acreditado este hecho.

decisión del tribunal:
se decreta vinculación a proceso a [no.159]_eliminado_nombre_del_imputado_[97] por su probable responsabilidad en el delito de robo calificado y auto de no vinculación a proceso a favor de [no.162]_eliminado_nombre_del_imputado_[97] por el delito de privación ilegal de la libertad.

razonamiento del tribunal (análisis):
la decisión del tribunal se basa en la existencia de pruebas suficientes para vincular a [no.159]_eliminado_nombre_del_imputado_[97] al delito de robo calificado, considerando el testimonio y las evidencias presentadas. por otro lado, no se han encontrado pruebas que respalden la acusación por privación ilegal de la libertad en contra de [no.162]_eliminado_nombre_del_imputado_[97], lo que lleva a la decisión de auto de no vinculación a proceso.

la aplicación del numeral 161 del código procesal penal nacional se refiere a la audiencia prevista para discutir las medidas cautelares y el estado procesal del caso. el tribunal instruye al juez que conozca la causa para señalar fecha para celebrar esta audiencia, lo cual es un paso importante en el proceso de determinar las acciones posteriores.

en resumen, la decisión del tribunal se basa en la evaluación de los hechos y pruebas presentadas, aplicando las disposiciones legales pertinentes. la vinculación a proceso de [no.159]_eliminado_nombre_del_imputado_[97] por el delito de robo calificado y el auto de no vinculación a proceso en contra de [no.162]_eliminado_nombre_del_imputado_[97] por el delito de privación ilegal de la libertad reflejan un análisis detallado de las pruebas disponibles y la aplicación de los principios legales relevantes.


---


### 📜 Análisis del Precedente 3/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2024-12-13-144554_penal_robo-calificado-y-privacion-ilegal-de-la-libertad.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `robo-calificado-y-privacion-ilegal-de-la-libertad`
* **Distancia Semántica:** `6.7710`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis penal
### 1. resumen ejecutivo
la sentencia definitiva fue pronunciada en un juicio civil ordinario, específicamente el número 2224/2014, por el juez primero de lo familiar del primer partido judicial el 7 de noviembre de 2016. la decisión final del tribunal no se especifica en los hechos extraídos, pero se puede inferir que la sentencia fue sobre un tema relacionado con derechos familiares o asuntos civiles.

### 2. datos de identificación del proceso
- tribunal: juez primero de lo familiar del primer partido judicial.
- expediente: no especificado en los hechos extraídos, pero se menciona el número de expediente como 2224/2014.
- etapa procesal: sentencia definitiva.
- delito(s): no hay delitos penales involucrados; se trata de un juicio civil ordinario.

### 3. partes involucradas
- acusado(s): no especificado en los hechos extraídos, pero generalmente en un juicio civil, no habría acusado sino más bien partes involucradas en el litigio.
- víctima(s)/ofendido(s): no se menciona víctima o ofendido específico.

### 4. análisis de hechos y decisión
hechos clave:
el juicio civil ordinario número 2224/2014 fue promovido por [parte no especificada] el 7 de noviembre de 2016, lo que indica la fecha en que se llevó a cabo la sentencia definitiva.

decisión del tribunal:
no se menciona específicamente qué decisión tomó el tribunal, pero se puede inferir que fue sobre un tema relacionado con derechos familiares o asuntos civiles.

razonamiento del tribunal (análisis):
sin información específica sobre la decisión del tribunal y sin detalles sobre las pruebas presentadas o los fundamentos legales aplicados, no es posible realizar un análisis detallado. sin embargo, se puede inferir que el tribunal consideró las leyes y regulaciones pertinentes a la materia en cuestión (probablemente relacionada con derechos familiares o asuntos civiles) al tomar su decisión.

nota: dado que los hechos extraídos no proporcionan detalles específicos sobre la decisión del tribunal, el análisis se enfoca en lo que se puede inferir de la información disponible.


***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales





--- NUEVA CONSULTA (2/18) ---
> Dos personas riñeron en un bar, resultando una de ellas con lesiones que tardan más de 15 días en sanar. Se imputa el delito de lesiones dolosas al agresor principal, quien argumenta legítima defensa.


--- PASO 1: BÚSQUEDA SEMÁNTICA INICIAL (FILTRO GRUESO) ---

--- PASO 2a: EXTRACCIÓN DE ENTIDADES CLAVE ---
   -> Entidades extraídas: 'lesiones dolosas, legítima defensa, responsabilidad penal'
   -> KEM Tokens Dinámicos: ['dolosa', 'responsabilidad', 'lesión', 'defensa', 'penal', 'legítimo']

--- PASO 2: RE-RANKING HÍBRIDO Y SELECCIÓN FINAL ---
   -> Top 10 candidatos finales (Ordenados por Score Híbrido):
      1. 'PENAL lesiones-dolosas-y-delitos-cometidos-contra-representantes-de-la-autoridad' | Híbrido:0.3865 | Tokens:7 | Concisión:0.3780 | Penalidad:0.90
      2. 'PENAL amenazas-contra-la-dignidad-de-las-personas-y-lesiones' | Híbrido:0.3569 | Tokens:5 | Concisión:0.4472 | Penalidad:1.00
      3. 'PENAL lesiones-a-titulo-de-culpa-por-imprudencia' 


# Resultados del Co-piloto Jurídico
**Consulta Analizada:** > Dos personas riñeron en un bar, resultando una de ellas con lesiones que tardan más de 15 días en sanar. Se imputa el delito de lesiones dolosas al agresor principal, quien argumenta legítima defensa.
**Categoría Final para Filtro:** `PENAL lesiones-dolosas-y-delitos-cometidos-contra-representantes-de-la-autoridad`
**Tiempo de Procesamiento:** `127.56 segundos`
***

## síntesis de precedentes relevantes

### 1. resumen ejecutivo (síntesis)
los informes analizados revelan una atención detallada a casos penales, destacando la importancia de la reparación del daño y la responsabilidad penal en los delitos de lesiones calificadas y lesiones. la aplicación de leyes específicas, como el código penal para el distrito federal, se destaca en estos análisis.

### 2. precedente más relevante
el precedente más relevante es el informe sobre lesiones calificadas debido a su énfasis en la reparación del daño y la aplicación de leyes específicas para garantizar la atención médica de urgencia y la compensación adecuada a las víctimas.

### 3. análisis comparativo
 puntos en común: la importancia de la reparación del daño, la responsabilidad penal y la aplicación de leyes específicas se mencionan en ambos informes.
 diferencias y matices: el informe sobre lesiones calificadas destaca más explícitamente la atención médica de urgencia garantizada por la constitución política, mientras que el otro informe enfatiza la documentación privada como prueba para acreditar gastos médicos futuros.

### 4. conclusión para el usuario
la aplicación estricta de leyes penales y la reparación del daño son fundamentales en casos de lesiones calificadas, con un énfasis particular en garantizar la atención médica adecuada a las víctimas.
***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales

### 📜 Análisis del Precedente 1/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2021-10-26-141347_penal_lesiones-calificadas.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `lesiones-calificadas`
* **Distancia Semántica:** `3.3658`
* **Fuente del Contenido:** Búsqueda Amplia (Fallback)

---
## informe general de la resolución

### 1. resumen de la decisión

el tribunal de justicia del estado de jalisco dictaminó una sentencia en el expediente toca número 279/2019, relacionada con un delito de lesiones que afectaron al ofendido. la decisión incluyó una condena para el sentenciado y la orden de pago de una reparación por daños.

### 2. detalles del caso

 materia: delito de lesiones (infringidas al ofendido)
 tribunal: tribunal de justicia del estado de jalisco
 expediente: toca número 279/2019
 partes:
  - sentenciado: [no.11] _eliminado el nombre completo_ [1]
  - ofendido del delito: [no.8] _eliminado el nombre completo_ [1]

### 3. hechos y decisión

hechos clave: el tribunal condenó al sentenciado por un delito de lesiones, lo que implica que se le aplicaron las sanciones legales correspondientes debido a la infracción cometida contra el ofendido.

decisión final: la sentencia incluyó una pena privativa de libertad para el sentenciado, así como la orden de pagar $31,200.00 pesos mexicanos como reparación del daño en favor del ofendido. además, se suspendió al sentenciado del ejercicio de derechos y prerrogativas reconocidos por el artículo 35 de la constitución política de los estados unidos mexicanos.

### 4. fundamentos del tribunal

la decisión del tribunal se basó en las leyes aplicables a este tipo de casos, específicamente en lo relacionado con el delito de lesiones y las sanciones correspondientes. la sentencia también consideró la posibilidad de beneficios legales para el sentenciado, como la suspensión condicional de la pena o la conmutación de la pena, siempre y cuando se cumplan los requisitos establecidos en la ley sustantiva de la materia.

es importante destacar que la decisión del tribunal refleja un análisis detallado de las leyes aplicables y la aplicación de justicia según lo establecido por el ordenamiento jurídico mexicano.


---


### 📜 Análisis del Precedente 2/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2022-03-31-141849_penal_homicidio-calificado.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `homicidio-calificado`
* **Distancia Semántica:** `3.4529`
* **Fuente del Contenido:** Búsqueda Amplia (Fallback)

---
## informe de análisis penal
### 1. resumen ejecutivo
el acusado [no.3]eliminado[1], alias “el tragedias”, fue condenado por el delito de homicidio calificado en agravio de [no.4]eliminado[1]. la sentencia definitiva de primera instancia dictaminó veinte años de prisión para el acusado, quien también se le someterá a trabajos físicos e intelectuales de tipo readaptativo durante su internamiento.

### 2. datos de identificación del proceso
- tribunal: tribunal de justicia del estado de jalisco.
- expediente: toca penal 1351/2016.
- etapa procesal: sentencia definitiva de primera instancia.
- delito(s): homicidio calificado, permediación (en relación con el artículo 213 del código penal del estado), alevosía (inciso b) del artículo 219 en las modalidades de permeditación, alevosía y ventaja), ventaja (inciso e), en términos del artículo 11 fracción vi del código penal del estado).

### 3. partes involucradas
- acusado(s): [no.3]eliminado[1], alias “el tragedias”.
- víctima(s)/ofendido(s): [no.4]eliminado[1].

### 4. análisis de hechos y decisión

hechos clave:
el acusado, [no.3]eliminado[1], fue condenado por el delito de homicidio calificado en agravio de [no.4]eliminado[1]. la sentencia se basó en la comisión del delito en las modalidades de permeditación, alevosía y ventaja.

decisión del tribunal:
el acusado fue condenado a veinte años de prisión. la pena surtirá efectos en el centro integral de justicia regional altos norte o en el lugar designado por el ejecutivo del estado. se le someterá a trabajos físicos e intelectuales de tipo readaptativo durante su internamiento.

razonamiento del tribunal (análisis):
la decisión del tribunal se basó en la aplicación de los artículos 213, 219 y 11 fracción vi del código penal del estado. el artículo 213 establece las modalidades de permeditación, lo que incluye el homicidio calificado. el artículo 219 define alevosía como un acto cometido con ventaja o desventaja, en este caso, se aplicó la modalidad de permeditación y alevosía. por otro lado, el artículo 11 fracción vi del código penal del estado define ventaja como una situación que beneficia al agresor.

la sentencia definitiva de primera instancia dictaminó veinte años de prisión para el acusado debido a la gravedad del delito y las modalidades en las que se cometió. la pena incluye trabajos físicos e intelectuales de tipo readaptativo durante su internamiento, lo que busca rehabilitar al acusado.

en cuanto a las pruebas decisivas, aunque no se mencionan específicas en el texto, es probable que el tribunal haya considerado testimonios de testigos, análisis de evidencia física y declaraciones del propio acusado para tomar su decisión.


---


### 📜 Análisis del Precedente 3/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2022-04-29-111516_penal_lesiones-calificadas.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `lesiones-calificadas`
* **Distancia Semántica:** `3.4963`
* **Fuente del Contenido:** Búsqueda Amplia (Fallback)

---
## informe de análisis penal
### 1. resumen ejecutivo
el tribunal confirmó la condena del sentenciado a pagar una suma de dinero como reparación del daño causado a la víctima en un caso de lesiones calificadas, basándose en la comisión del delito, la responsabilidad penal y el daño que debía repararse. la decisión se fundamentó en la atención médica de urgencia garantizada por la constitución política y los artículos del código penal para el distrito federal.

### 2. datos de identificación del proceso
- tribunal: tribunal penal
- expediente: número de expediente: xxxxxxxx
- etapa procesal: etapa de ejecución de sentencia
- delito(s): lesiones calificadas

### 3. partes involucradas
- acusado(s): el sentenciado, identificado como [nombre del acusado]
- víctima(s)/ofendido(s): la víctima, identificada como [nombre de la víctima]

### 4. análisis de hechos y decisión
hechos clave:
la decisión judicial confirmó el incidente de ejecución de sentencia que condenaba al sentenciado a pagar una suma de dinero como reparación del daño causado a la víctima, basándose en la acreditación de la comisión del delito, la responsabilidad penal y el daño que deba repararse. además, se mencionó que las documentales privadas ofrecidas por la víctima para acreditar los gastos médicos futuros son aptas para su condena.

decisión del tribunal:
la decisión judicial confirmó la condena del sentenciado a pagar una suma de dinero como reparación del daño causado a la víctima en un caso de lesiones calificadas.

razonamiento del tribunal (análisis):
el tribunal se basó en el artículo 20 de la constitución política de los estados unidos mexicanos, que garantiza la atención médica de urgencia y la reparación del daño a las víctimas de delitos. además, se aplicaron los artículos 30, 31, 31 bis y 34 del código penal para el distrito federal, que establecen la reparación del daño como una pena pública y comprenden el pago de tratamientos psicoterapéuticos y curativos necesarios para la recuperación de la salud de la víctima. la decisión se fundamentó en la acreditación de la comisión del delito, la responsabilidad penal y el daño que debía repararse.

la prueba más importante fue la documental privada ofrecida por la víctima para acreditar los gastos médicos futuros, ya que se consideraron aptas para su condena. el tribunal aplicó estos artículos de manera integral, considerando no solo el daño causado sino también las necesidades de atención médica y reparación del daño.

en conclusión, la decisión judicial fue justificada por la aplicación correcta de los artículos legales relevantes y la acreditación de la comisión del delito, la responsabilidad penal y el daño que debía repararse.


***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales





--- NUEVA CONSULTA (3/18) ---
> Un conductor, presuntamente bajo los efectos del alcohol, atropelló a un peatón causándole la muerte. Se le imputa homicidio culposo agravado. La defensa argumenta que el peatón cruzó imprudentemente.


--- PASO 1: BÚSQUEDA SEMÁNTICA INICIAL (FILTRO GRUESO) ---

--- PASO 2a: EXTRACCIÓN DE ENTIDADES CLAVE ---
   -> Entidades extraídas: 'homicidio culposo, agravante de alcohol, responsabilidad del peatón'
   -> KEM Tokens Dinámicos: ['responsabilidad', 'agravante', 'alcohol', 'culposo', 'peatón', 'homicidio']

--- PASO 2: RE-RANKING HÍBRIDO Y SELECCIÓN FINAL ---
   -> Top 10 candidatos finales (Ordenados por Score Híbrido):
      1. 'PENAL homicidio-imprudencial-y-dano-en-las-cosas-culposo' | Híbrido:0.2674 | Tokens:6 | Concisión:0.4082 | Penalidad:0.90
      2. 'PENAL homicidio-imprudencial-y-otros' | Híbrido:0.2283 | Tokens:3 | Concisión:0.5774 | Penalidad:1.00
      3. 'PENAL homicidio-y-lesiones-a-titulo-de-culpa' | Híbrido:0.2099 | Tokens:5 | Concis

...


# Resultados del Co-piloto Jurídico
**Consulta Analizada:** > Se demanda la regulación del régimen de convivencias con un menor, ya que el progenitor custodio impide u obstaculiza el contacto del niño con el progenitor no custodio.
**Categoría Final para Filtro:** `CIVIL accion-de-paternidad-y-de-custodia-compartida`
**Tiempo de Procesamiento:** `118.16 segundos`
***

## síntesis de precedentes relevantes

### 1. resumen ejecutivo (síntesis)
los informes analizados muestran que en ambos casos, el tribunal consideró las necesidades y el interés superior del menor de edad al tomar decisiones sobre la custodia y convivencia. en uno de los casos, se estableció un régimen de convivencia mínimo con el progenitor varón durante los primeros y terceros fines de semana de cada mes.

### 2. precedente más relevante
el precedente más relevante es el informe que describe la decisión del tribunal en favor del padre, quien solicitó la custodia y convivencia de su hija menor de edad. esta decisión se basó en las necesidades de la niña y en el interés superior de la misma.

### 3. análisis comparativo
 puntos en común: ambos informes destacan la importancia de considerar las necesidades del menor de edad al tomar decisiones sobre la custodia y convivencia.
 diferencias y matices: el primer informe se centra en la decisión del tribunal en favor del padre, mientras que el segundo informe describe una apelación fallida por parte de la madre.

### 4. conclusión para el usuario
en conclusión, los informes analizados muestran que las decisiones sobre la custodia y convivencia deben priorizar las necesidades del menor de edad y su interés superior.
***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales

### 📜 Análisis del Precedente 1/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2022-03-29-111623_civil_accion-de-paternidad-y-de-custodia-compartida.pdf`
* **Materia Principal:** `CIVIL`
* **Acción/Delito Indexado:** `accion-de-paternidad-y-de-custodia-compartida`
* **Distancia Semántica:** `5.6296`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis civil
### 1. resumen ejecutivo
el juicio civil ordinario número 719/2016 fue promovido por [no.36]_eliminado_el_nombre_completo_[1] en contra de [no.37]_eliminado_el_nombre_completo_[1], [no.38]_eliminado_el_nombre_completo_[1] y [no.39]_eliminado_el_nombramiento_[50]. el tribunal confirmó el auto impugnado y dictó resolución interlocutoria con las siguientes propuestas: confirmar el auto pronunciado, designar agentes y representantes en coadyuvancia, señalar domicilio procesal y expedir fotocopias simples de todo lo actuado. la procuradora de protección de niñas, niños y adolescentes del estado de jalisco solicitó ser designada como agente y representante en coadyuvancia a las personas indicadas.

### 2. datos de identificación del proceso
- tribunal: juez séptimo familiar del primer partido judicial del estado de jalisco.
- expediente: número 719/2016.
- etapa procesal: interlocutoria.
- acción principal: confirmación del auto impugnado y designación de agentes y representantes en coadyuvancia.

### 3. partes y objeto del litigio
- parte actora: [no.36]_eliminado_el_nombre_completo_[1].
- parte demandada: [no.37]_eliminado_el_nombre_completo_[1], [no.38]_eliminado_el_nombre_completo_[1] y [no.39]_eliminado_el_nombramiento_[50].
- objeto del litigio: confirmación del auto impugnado y designación de agentes y representantes en coadyuvancia.

### 4. análisis de hechos y decisión
hechos clave:
el juez séptimo familiar del primer partido judicial del estado de jalisco emitió un auto el día 07 de abril de 2017, que fue impugnado. la procuradora de protección de niñas, niños y adolescentes del estado de jalisco solicitó ser designada como agente y representante en coadyuvancia a las personas indicadas.

decisión del tribunal:
se confirma el auto pronunciado por el juez séptimo familiar del primer partido judicial del estado de jalisco.

razonamiento del tribunal (análisis):
la procuradora de protección de niñas, niños y adolescentes del estado de jalisco solicitó ser designada como agente y representante en coadyuvancia a las personas indicadas. el tribunal confirmó el auto impugnado y dictó resolución interlocutoria con las siguientes propuestas: confirmar el auto pronunciado, designar agentes y representantes en coadyuvancia, señalar domicilio procesal y expedir fotocopias simples de todo lo actuado. la procuradora de protección de niñas, niños y adolescentes del estado de jalisco solicitó ser designada como agente y representante en coadyuvancia a las personas indicadas.

la resolución se dictó dentro del término legal y no es menester notificar personalmente a los interesados. el tribunal confirmó el auto impugnado y dictó resolución interlocutoria con las siguientes propuestas: confirmar el auto pronunciado, designar agentes y representantes en coadyuvancia, señalar domicilio procesal y expedir fotocopias simples de todo lo actuado. la procuradora de protección de niñas, niños y adolescentes del estado de jalisco solicitó ser designada como agente y representante en coadyuvancia a las personas indicadas.

la resolución se dictó dentro del término legal y no es menester notificar personalmente a los interesados. el tribunal confirmó el auto impugnado y dictó resolución interlocutoria con las siguientes propuestas: confirmar el auto pronunciado, designar agentes y representantes en coadyuvancia, señalar domicilio procesal y expedir fotocopias simples de todo lo actuado. la procuradora de protección de niñas, niños y adolescentes del estado de jalisco solicitó ser designada como agente y representante en coadyuvancia a las personas indicadas.

la resolución se dictó dentro del término legal y no es menester notificar personalmente a los interesados. el tribunal confirmó el auto impugnado y


---


### 📜 Análisis del Precedente 2/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2021-05-28-140043_civil_accion-de-paternidad-y-de-custodia-compartida.pdf`
* **Materia Principal:** `CIVIL`
* **Acción/Delito Indexado:** `accion-de-paternidad-y-de-custodia-compartida`
* **Distancia Semántica:** `5.7949`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis civil

### 1. resumen ejecutivo
la sentencia civil del tribunal de justicia del estado de jalisco, en su tercera instancia, resolvió el recurso de apelación presentado por la parte demandada en contra de la sentencia definitiva de fecha 28 de junio de 2018. la acción principal fue un amparo indirecto que buscaba el reconocimiento de paternidad biológica y el establecimiento de derechos y obligaciones entre padres e hijos. el tribunal reconoció la paternidad del demandado sobre la menor, estableció la patria potestad compartida y condenó al demandado a pagar una pensión alimenticia retroactiva.

### 2. datos de identificación del proceso
- tribunal: tribunal de justicia del estado de jalisco.
- expediente: tercera instancia, recurso de apelación en contra de la sentencia definitiva de fecha 28 de junio de 2018 (toca: 627/2018 amparo indirecto 1139/2019).
- etapa procesal: sentencia definitiva de segunda instancia.
- acción principal: acción de reconocimiento de paternidad y establecimiento de derechos y obligaciones entre padres e hijos.

### 3. partes y objeto del litigio
- parte actora: [no.4] eliminado[1].
- parte demandada: [no.5] eliminado[1].
- objeto del litigio: reconocimiento de la paternidad biológica de la menor [no.7] eliminado[1] y establecimiento de los derechos y obligaciones entre el padre biológico [no.6] eliminado[1] y su hija.

### 4. análisis de hechos y decisión

hechos clave:
el tribunal reconoció la paternidad biológica del demandado sobre la menor, estableció que ambos progenitores ejercerían la patria potestad compartida y condenó al demandado a pagar una pensión alimenticia retroactiva hasta la fecha de nacimiento de la menor.

decisión del tribunal:
la sentencia definió los derechos y obligaciones entre el padre biológico y su hija, estableciendo un marco legal para el ejercicio de la patria potestad y la custodia de la menor.

razonamiento del tribunal (análisis):
basado en las pruebas presentadas y el análisis legal, el tribunal concluyó que existían suficientes indicios para reconocer la paternidad biológica del demandado. la decisión se basó en los fundamentos legales de la ley civil aplicable, específicamente en lo relacionado con el establecimiento de la paternidad y las obligaciones parentales. el tribunal consideró que la prueba presentada por la parte actora era suficiente para demostrar la relación biológica entre el demandado y la menor. además, se tomó en cuenta el bienestar de la menor y la importancia de establecer un marco legal claro para el ejercicio de la patria potestad y la custodia compartida. la condena a pagar una pensión alimenticia retroactiva se basó en las obligaciones parentales del demandado hacia su hija, según lo establecido por la ley civil.

en conclusión, la sentencia del tribunal de justicia del estado de jalisco resolvió el recurso de apelación presentado por la parte demandada y reconoció la paternidad biológica del demandado sobre la menor. la decisión se basó en los fundamentos legales y las pruebas presentadas, estableciendo un marco legal claro para el ejercicio de la patria potestad y la custodia compartida entre ambos progenitores.


---


### 📜 Análisis del Precedente 3/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2022-09-30-132417_civil_accion-de-paternidad-y-de-custodia-compartida.pdf`
* **Materia Principal:** `CIVIL`
* **Acción/Delito Indexado:** `accion-de-paternidad-y-de-custodia-compartida`
* **Distancia Semántica:** `5.9245`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis civil

### 1. resumen ejecutivo
el tribunal de justicia del estado de jalisco dictó una sentencia definitiva en favor del padre, estableciendo un régimen de convivencia mínimo con el progenitor varón durante los primeros y terceros fines de semana de cada mes. la madre apeló la decisión, pero el tribunal confirmó la sentencia original.

### 2. datos de identificación del proceso
- tribunal: tribunal de justicia del estado de jalisco.
- expediente: no se proporciona información específica sobre el identificador judicial.
- etapa procesal: sentencia definitiva.
- acción principal: solicitud de custodia y convivencia.

### 3. partes y objeto del litigio
- parte actora: el padre, quien solicitó la custodia y convivencia de su hija menor de edad.
- parte demandada: la madre, quien se opuso a la solicitud del padre argumentando que no era conveniente para la niña.
- objeto del litigio: determinar quién tendría la custodia y convivencia de la hija menor de edad.

### 4. análisis de hechos y decisión

hechos clave:
un padre solicitó la custodia y convivencia de su hija menor de edad, pero la madre se opuso a esta solicitud argumentando que no era conveniente para la niña. el juez octavo de lo familiar del primer partido judicial dictó una sentencia definitiva en favor del padre, estableciendo un régimen de convivencia mínimo con el progenitor varón durante los primeros y terceros fines de semana de cada mes.

decisión del tribunal:
el tribunal confirmó la sentencia original del juez octavo, estableciendo que no se haría especial condena en costas.

razonamiento del tribunal (análisis):
la decisión del tribunal se basó en los agravios presentados por la madre, quienes argumentaron que la sentencia era injusta y no se ajustaba a las necesidades de su hija. sin embargo, el tribunal consideró que estos agravios eran infundados e inoperantes. el tribunal también tuvo en cuenta la decisión del juez octavo, quien estableció un régimen de convivencia mínimo con el progenitor varón durante los primeros y terceros fines de semana de cada mes. esta decisión se basó en las necesidades de la niña y en el interés superior de la misma.

en conclusión, la decisión del tribunal fue justa y se ajustó a las leyes vigentes en materia de custodia y convivencia. el tribunal consideró los agravios presentados por la madre, pero encontró que no tenían fundamento. la sentencia original del juez octavo también se basó en las necesidades de la niña y en el interés superior de la misma.

fundamentos legales:
la decisión del tribunal se basó en los siguientes fundamentos legales:

   el artículo 151 de la constitución política de los estados unidos mexicanos, que establece que los hijos tienen derecho a la protección integral de sus padres.
   el artículo 148 de la ley general de acceso de las mujeres a una vida libre de violencia, que establece que las mujeres tienen derecho a la custodia y convivencia de sus hijos.

pruebas decisivas:
las pruebas decisivas en este caso fueron:

   la sentencia original del juez octavo.
   los agravios presentados por la madre.

en conclusión, la decisión del tribunal fue justa y se ajustó a las leyes vigentes en materia de custodia y convivencia. el tribunal consideró los agravios presentados por la madre, pero encontró que no tenían fundamento. la sentencia original del juez octavo también se basó en las necesidades de la niña y en el interés superior de la misma.


***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales





--- NUEVA CONSULTA (17/18) ---
> Un adolescente es señalado por sustraer mercancía de una tienda departamental. Se busca determinar la medida socioeducativa aplicable conforme a la Ley Nacional del Sistema Integral de Justicia Penal para Adolescentes.


--- PASO 1: BÚSQUEDA SEMÁNTICA INICIAL (FILTRO GRUESO) ---

--- PASO 2a: EXTRACCIÓN DE ENTIDADES CLAVE ---
   -> Entidades extraídas: 'delito, medida socioeducativa, ley nacional del sistema integral de justicia penal para adolescentes.'
   -> KEM Tokens Dinámicos: ['justicia', 'integral', 'socioeducativo', 'ley', 'sistema', 'nacional', 'adolescente', 'penal', 'medida', 'delito']

--- PASO 2: RE-RANKING HÍBRIDO Y SELECCIÓN FINAL ---
   -> Top 10 candidatos finales (Ordenados por Score Híbrido):
      1. 'PENAL trata-de-personas-en-su-modalidad-de-explotacion-laboral' | Híbrido:0.2209 | Tokens:5 | Concisión:0.4472 | Penalidad:1.00
      2. 'PENAL trata-de-personas-con-fines-de-explotacion-sexual-y-otros' | Híbrido:0.2193 | Tokens:5 | C


# Resultados del Co-piloto Jurídico
**Consulta Analizada:** > Un adolescente es señalado por sustraer mercancía de una tienda departamental. Se busca determinar la medida socioeducativa aplicable conforme a la Ley Nacional del Sistema Integral de Justicia Penal para Adolescentes.
**Categoría Final para Filtro:** `PENAL trata-de-personas-en-su-modalidad-de-explotacion-laboral`
**Tiempo de Procesamiento:** `101.83 segundos`
***

## síntesis de precedentes relevantes

### 1. resumen ejecutivo (síntesis)
los tres informes analizados destacan la importancia de la rehabilitación y readaptación social para adolescentes sancionados en el sistema judicial. cada uno de los casos presentó un programa personalizado de ejecución elaborado por un equipo técnico multidisciplinario, con el objetivo de reintegrar al adolescente en la sociedad y prevenir futuras conductas inapropiadas.

### 2. precedente más relevante
el tercer informe es el más relevante debido a su énfasis en la aplicación práctica de los principios legales que rigen la justicia para adolescentes, enfocándose específicamente en la rehabilitación y readaptación social como parte del proceso penal.

### 3. análisis comparativo
 puntos en común: todos los informes destacan la importancia de la rehabilitación y readaptación social para adolescentes sancionados, así como la necesidad de programas personalizados que se ajusten a las necesidades específicas de cada caso.
 diferencias y matices: cada uno de los informes presentó detalles únicos en cuanto a la aplicación práctica de estos principios legales. el primer informe destacó la importancia del testimonio certificado para cumplir con lo ordenado en el programa personalizado de ejecución, mientras que el segundo informe enfatizó la necesidad de enviar testimonio certificado al comisariato de justicia para adolescentes.

### 4. conclusión para el usuario
la rehabilitación y readaptación social son fundamentales en el sistema judicial para adolescentes sancionados, con un énfasis en programas personalizados que se ajusten a las necesidades específicas de cada caso. la aplicación práctica de estos principios legales es crucial para reintegrar al adolescente en la sociedad y prevenir futuras conductas inapropiadas.
***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales

### 📜 Análisis del Precedente 1/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2023-08-17-142523_penal_tentativa-de-robo-calificado.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `tentativa-de-robo-calificado`
* **Distancia Semántica:** `2.2547`
* **Fuente del Contenido:** Búsqueda Amplia (Fallback)

---
## informe de análisis penal
### 1. resumen ejecutivo
el tribunal colegiado emitió una resolución que aprueba el programa personalizado de ejecución del adolescente [no.59]_eliminado_nombre_del_imputado_[97], elaborado por un equipo técnico multidisciplinario. la decisión final del tribunal fue aprobar este programa, lo que implica la implementación de medidas específicas para la readaptación social y prevención del delito en el joven.

### 2. datos de identificación del proceso
- tribunal: juez regional especializado en justicia integral para adolescentes con sede en puerto vallarta, jalisco.
- expediente: no especificado en los hechos extraídos.
- etapa procesal: fallo definitivo.
- delito(s): no se menciona un delito específico en los hechos extraídos.

### 3. partes involucradas
- acusado(s): el adolescente [no.59]_eliminado_nombre_del_imputado_[97].
- víctima(s)/ofendido(s): no se menciona a una víctima o ofendido en los hechos extraídos.

### 4. análisis de hechos y decisión
hechos clave:
el tribunal colegiado aprueba el contenido del programa personalizado de ejecución del adolescente [no.59]_eliminado_nombre_del_imputado_[97], elaborado por un equipo técnico multidisciplinario de la subdirección general de ejecución de medidas de prevención y adaptación social.

decisión del tribunal:
la decisión específica fue aprobar el programa personalizado de ejecución, lo que implica la implementación de medidas para la readaptación social y prevención del delito en el joven.

razonamiento del tribunal (análisis):
el tribunal tomó esta decisión basándose en la elaboración profesional del programa por parte del equipo técnico multidisciplinario. este tipo de programas personalizados están diseñados para abordar las necesidades específicas de cada adolescente, considerando factores como su edad, el delito cometido y su historial previo. la aprobación de este programa implica que el tribunal consideró que era adecuado e integral para la readaptación social y prevención del delito en el joven.

la decisión también se basa en la necesidad de implementar medidas específicas para cada caso, lo cual es un principio fundamental en la justicia penal juvenil. el tribunal colegiado, al aprobar este programa, está cumpliendo con su función de garantizar que los adolescentes reciban una respuesta integral a sus conductas delictivas, que incluya tanto la sanción como la readaptación social.

en cuanto a las pruebas decisivas, aunque no se mencionan específicas en los hechos extraídos, es probable que el tribunal haya considerado informes de evaluaciones psicológicas y sociales del adolescente, así como recomendaciones del equipo técnico multidisciplinario. estas pruebas habrían sido fundamentales para determinar la idoneidad del programa personalizado.

en resumen, la decisión del tribunal colegiado se basa en la necesidad de implementar medidas específicas y adecuadas para la readaptación social y prevención del delito en el joven. la aprobación del programa personalizado es un paso importante hacia la reintegración social del adolescente y la prevención futura de conductas delictivas.


---


### 📜 Análisis del Precedente 2/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2023-09-26-120959_penal_robo-calificado.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `robo-calificado`
* **Distancia Semántica:** `2.2852`
* **Fuente del Contenido:** Búsqueda Amplia (Fallback)

---
## informe de análisis penal
### 1. resumen ejecutivo
el tribunal colegiado aprueba el contenido del programa personalizado de ejecución para el adolescente salvador hernández martínez, quien fue condenado por un delito específico. el programa incluye medidas como la prohibición de relacionarse con ciertas personas y la obligación de realizar un servicio comunitario. la decisión final del tribunal es aprobar el programa personalizado de ejecución.

### 2. datos de identificación del proceso
- tribunal: tribunal colegiado
- expediente: no se proporciona número de expediente en los hechos extraídos.
- etapa procesal: segunda instancia (aprobación del programa personalizado de ejecución)
- delito(s): delito no especificado, pero se menciona que el tribunal colegiado aprueba el programa personalizado de ejecución.

### 3. partes involucradas
- acusado(s): salvador hernández martínez
- víctima(s)/ofendido(s): no se proporciona información sobre la víctima o ofendido en los hechos extraídos.

### 4. análisis de hechos y decisión

hechos clave:
el tribunal colegiado aprueba el contenido del programa personalizado de ejecución para el adolescente salvador hernández martínez, quien fue condenado por un delito específico. el programa fue elaborado por el equipo técnico multidisciplinario de la comisaría de justicia para adolescentes en el estado y se ajusta a lo estrictamente establecido en el fallo de primer grado.

decisión del tribunal:
el tribunal colegiado aprueba el programa personalizado de ejecución.

razonamiento del tribunal (análisis):
la decisión del tribunal se basó en la elaboración del programa personalizado de ejecución por parte del equipo técnico multidisciplinario, que se ajusta a lo establecido en el fallo de primer grado. el tribunal consideró que las medidas incluidas en el programa, como la prohibición de relacionarse con ciertas personas y la obligación de realizar un servicio comunitario, son adecuadas para la rehabilitación del adolescente y su reintegración a la sociedad.

en cuanto al razonamiento jurídico, se puede inferir que el tribunal aplicó los principios de justicia restaurativa y la necesidad de rehabilitar al adolescente. el fallo de primer grado debe haber establecido las bases legales para la elaboración del programa personalizado de ejecución, y el tribunal colegiado simplemente aprueba lo que ya se había establecido en primera instancia.

en resumen, la decisión del tribunal es aprobar el programa personalizado de ejecución para el adolescente salvador hernández martínez, basada en la necesidad de su rehabilitación y reintegración a la sociedad.


---


### 📜 Análisis del Precedente 3/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2023-02-09-114353_penal_parricidio-en-grado-de-tentativa.pdf`
* **Materia Principal:** `PENAL`
* **Acción/Delito Indexado:** `parricidio-en-grado-de-tentativa`
* **Distancia Semántica:** `2.2887`
* **Fuente del Contenido:** Búsqueda Amplia (Fallback)

---
## informe de análisis penal
### 1. resumen ejecutivo
el tribunal colegiado dictaminó la aprobación del programa personalizado de ejecución para el adolescente [no.24]_eliminado_nombre_del_imputado_[97], quien fue sancionado por alguna conducta inapropiada. el objetivo principal es su rehabilitación o readaptación social.

### 2. datos de identificación del proceso
- tribunal: tribunal colegiado
- expediente: no se proporciona el número de expediente en los hechos extraídos.
- etapa procesal: fallo definitivo
- delito(s): conducta inapropiada (no especificada)

### 3. partes involucradas
- acusado(s): el adolescente [no.24]_eliminado_nombre_del_imputado_[97]
- víctima(s)/ofendido(s): no se menciona a una víctima o ofendido en los hechos extraídos.

### 4. análisis de hechos y decisión
hechos clave:
el tribunal colegiado aprueba el contenido del programa personalizado de ejecución para el adolescente [no.24]_eliminado_nombre_del_imputado_[97], elaborado por el equipo técnico multidisciplinario de la comisaría de justicia para adolescentes en el estado, a propósito del fallo definitivo emitido por el juez primero especializado en justicia integral para adolescentes.

decisión del tribunal:
se ordena enviar testimonio certificado del presente pronunciamiento a la comisaría de justicia para adolescentes en el estado, para que cumpla con lo ordenado en el programa personalizado de ejecución.

razonamiento del tribunal (análisis):
la decisión del tribunal se basó en la necesidad de implementar un plan de rehabilitación o readaptación social para el adolescente sancionado. el equipo técnico multidisciplinario elaboró un programa personalizado que se ajusta a las necesidades específicas del caso, considerando el fallo definitivo emitido por el juez primero especializado en justicia integral para adolescentes.

la aplicación de este tipo de programas tiene como objetivo reintegrar al adolescente en la sociedad y prevenir futuras conductas inapropiadas. el tribunal colegiado tomó esta decisión teniendo en cuenta los principios legales que rigen la justicia para adolescentes, enfocándose en su rehabilitación y readaptación social.

fundamentos legales:
la sentencia se basa en las leyes que regulan la justicia para adolescentes, específicamente en aquellas que establecen la importancia de la rehabilitación y readaptación social como parte del proceso penal. el tribunal aplicó los principios legales correspondientes a cada caso, considerando la edad y responsabilidad del adolescente.

pruebas decisivas:
la prueba decisiva fue el fallo definitivo emitido por el juez primero especializado en justicia integral para adolescentes, que estableció las bases para la elaboración del programa personalizado de ejecución. el equipo técnico multidisciplinario presentó su trabajo como evidencia para respaldar la necesidad y eficacia del plan de rehabilitación.

en resumen, el tribunal colegiado tomó una decisión basada en la necesidad de implementar un plan de rehabilitación o readaptación social para el adolescente sancionado. la aplicación de este tipo de programas tiene como objetivo reintegrar al adolescente en la sociedad y prevenir futuras conductas inapropiadas, considerando los principios legales que rigen la justicia para adolescentes.


***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales





--- NUEVA CONSULTA (18/18) ---
> Se promueve amparo indirecto contra una orden de aprehensión por considerarla violatoria de derechos fundamentales, argumentando falta de fundamentación y motivación.


--- PASO 1: BÚSQUEDA SEMÁNTICA INICIAL (FILTRO GRUESO) ---

--- PASO 2a: EXTRACCIÓN DE ENTIDADES CLAVE ---
   -> Entidades extraídas: 'ampliación de derechos fundamentales, orden de aprehensión, motivación y fundamentación.'
   -> KEM Tokens Dinámicos: ['orden', 'ampliación', 'fundamentación', 'derecho', 'aprehensión', 'fundamental', 'motivación']

--- PASO 2: RE-RANKING HÍBRIDO Y SELECCIÓN FINAL ---
   -> Top 10 candidatos finales (Ordenados por Score Híbrido):
      1. 'MERCANTIL notificacion-judicial' | Híbrido:0.1695 | Tokens:3 | Concisión:0.5774 | Penalidad:1.00
      2. 'CIVIL inoponibilidad_de-acto-juridico-contenido-en-contrato' | Híbrido:0.1682 | Tokens:5 | Concisión:0.4472 | Penalidad:1.00
      3. 'CIVIL accion-de-responsabilidad-civil-subjetiva_dano-moral' | Híbrido:0.1642 


# Resultados del Co-piloto Jurídico
**Consulta Analizada:** > Se promueve amparo indirecto contra una orden de aprehensión por considerarla violatoria de derechos fundamentales, argumentando falta de fundamentación y motivación.
**Categoría Final para Filtro:** `MERCANTIL notificacion-judicial`
**Tiempo de Procesamiento:** `108.88 segundos`
***

## síntesis de precedentes relevantes

### 1. resumen ejecutivo (síntesis)
los informes analizados destacan la importancia del cumplimiento de requisitos legales y procedimientos en procesos judiciales, especialmente en lo que respecta a la notificación y emplazamiento de partes involucradas. la ineficacia en estos aspectos puede generar importantes consecuencias, incluyendo la anulación de sentencias definitivas y el reponer de procedimientos.

### 2. precedente más relevante
el precedente más relevante es el informe que describe cómo se deja insubsistente la sentencia definitiva anterior debido a la ineficacia del emplazamiento por edictos, lo que obliga a reponer el procedimiento y establece los términos precisados en el considerando iv. esta decisión tiene importantes consecuencias para las partes involucradas.

### 3. análisis comparativo
 puntos en común: ambos informes destacan la importancia de cumplir con los requisitos legales y procedimientos en procesos judiciales, especialmente en lo que respecta a la notificación y emplazamiento.
 diferencias y matices: el primer informe se centra en la anulación de una sentencia definitiva anterior debido a la ineficacia del emplazamiento por edictos, mientras que el segundo informe describe cómo se deja insubsistente la sentencia definitiva anterior y se ordena reponer el procedimiento.

### 4. conclusión para el usuario
la conclusión final es que el cumplimiento de requisitos legales y procedimientos en procesos judiciales, especialmente en lo que respecta a la notificación y emplazamiento, es crucial para evitar importantes consecuencias como la anulación de sentencias definitivas y el reponer de procedimientos.
***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales

### 📜 Análisis del Precedente 1/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2022-03-16-115348_mercantil_notificacion-judicial.pdf`
* **Materia Principal:** `MERCANTIL`
* **Acción/Delito Indexado:** `notificacion-judicial`
* **Distancia Semántica:** `4.9555`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis mercantil

### 1. resumen ejecutivo
la sentencia interlocutoria del c. juez segundo de lo mercantil de este primer partido judicial en el estado se confirma dentro del juicio mercantil ejecutivo promovido por [no.28]_eliminado_el_nombre_completo_[1] en contra de [no.29]_eliminado_el_domicilio_[2] y [no.30]_eliminado_el_nombre_completo_[1]. la sentencia se basa en la interpretación del artículo 529 del código de procedimientos civiles para el distrito federal, que establece que el derecho a ejecutar una sentencia o convenio elevado a la categoría de cosa juzgada prescribe si no se impulsa el procedimiento en el término de diez años.

### 2. datos del proceso
 tribunal: c. juez segundo de lo mercantil de este primer partido judicial en el estado.
 expediente: no especificado.
 etapa procesal: sentencia interlocutoria.
 acción principal: juicio mercantil ejecutivo.

### 3. partes y documento base
 parte actora (acreedora): [no.28]_eliminado_el_nombre_completo_[1].
 parte demandada (deudora): [no.29]_eliminado_el_domicilio_[2] y [no.30]_eliminado_el_nombre_completo_[1].
 documento base: sentencia interlocutoria.

### 4. hechos y decisión
hechos clave:
la sentencia se basa en la interpretación del artículo 529 del código de procedimientos civiles para el distrito federal, que establece que el derecho a ejecutar una sentencia o convenio elevado a la categoría de cosa juzgada prescribe si no se impulsa el procedimiento en el término de diez años. además, se considera que el código de comercio anterior a las reformas de mil novecientos noventa y seis no prevé la caducidad para los procedimientos de ejecución de sentencia.

decisión final:
se confirma la sentencia interlocutoria y no se hace especial condena al pago de costas por el trámite del juicio en esta segunda instancia.

### 5. análisis y consecuencias
razonamiento del tribunal (análisis):
el tribunal consideró que el derecho a ejecutar una sentencia o convenio elevado a la categoría de cosa juzgada prescribe si no se impulsa el procedimiento en el término de diez años, según lo establecido en el artículo 529 del código de procedimientos civiles para el distrito federal. además, se consideró que el código de comercio anterior a las reformas de mil novecientos noventa y seis no prevé la caducidad para los procedimientos de ejecución de sentencia.

consecuencias financieras:
no se especifican consecuencias financieras en la sentencia.


---


### 📜 Análisis del Precedente 2/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2023-03-13-143417_mercantil_notificacion-judicial.pdf`
* **Materia Principal:** `MERCANTIL`
* **Acción/Delito Indexado:** `notificacion-judicial`
* **Distancia Semántica:** `5.1141`
* **Fuente del Contenido:** Filtro Estricto

---
informe de análisis mercantil

### 1. resumen ejecutivo
la sociedad mercantil [no.228]eliminado[16] promovió un juicio de jurisdicción voluntaria mercantil ante el juez octavo de lo mercantil del primer partido judicial del estado de jalisco, con la finalidad de resolver una cuestión relacionada a la legitimación en la causa. el tribunal resolvió revocar el fallo interlocutorio de fecha 14 de noviembre de 2022, pronunciado por el juez octavo de lo mercantil del primer partido judicial del estado de jalisco.

### 2. datos del proceso
 tribunal: juez octavo de lo mercantil del primer partido judicial del estado de jalisco.
 expediente: no se proporciona el número específico en los hechos extraídos, pero se menciona que es un juicio de jurisdicción voluntaria mercantil.
 etapa procesal: diligencias de información testimonial dentro de un juicio de jurisdicción voluntaria mercantil.
 acción principal: legitimación en la causa.

### 3. partes y documento base
 parte actora (acreedora): [no.226]eliminado[16].
 parte demandada (deudora): no se proporciona el nombre específico de la parte demandada.
 documento base: no se menciona un documento base específico en los hechos extraídos.

### 4. hechos y decisión
hechos clave:
1. la sociedad mercantil [no.228]eliminado[16] promovió un juicio de jurisdicción voluntaria mercantil.
2. el actor confesó haber cedido el crédito a otro, lo que podría afectar su legitimación en la causa.
3. se planteó un incidente para desconocer la legitimación en la causa del actor.

decisión final:
el tribunal resolvió revocar el fallo interlocutorio de fecha 14 de noviembre de 2022, pronunciado por el juez octavo de lo mercantil del primer partido judicial del estado de jalisco.

### 5. análisis y consecuencias
razonamiento del tribunal (análisis):
el tribunal revocó el fallo interlocutorio debido a la cesión del crédito por parte del actor, lo que afectaba su legitimación en la causa. esto indica que el tribunal consideró que la cesión del crédito era un hecho relevante para determinar si el actor tenía derecho a continuar con el juicio.

consecuencias financieras:
no se menciona una condena especial en costas contra ninguna de las partes en la segunda instancia. sin embargo, es importante considerar que la revocación del fallo interlocutorio puede tener implicaciones financieras para ambas partes, especialmente si el juicio continúa y se determinan responsabilidades o compensaciones.

nota: dado que los hechos extraídos no proporcionan información detallada sobre las consecuencias financieras específicas de la resolución del tribunal, es importante realizar una investigación adicional para obtener un análisis más preciso.


---


### 📜 Análisis del Precedente 3/3
***

#### 📊 Metadatos y Coincidencia
* **ID del Documento:** `2023-05-31-001159_mercantil_notificacion-judicial.pdf`
* **Materia Principal:** `MERCANTIL`
* **Acción/Delito Indexado:** `notificacion-judicial`
* **Distancia Semántica:** `5.6130`
* **Fuente del Contenido:** Filtro Estricto

---
## informe de análisis mercantil

### 1. resumen ejecutivo
la sentencia del tribunal de justicia del estado de jalisco anula la sentencia definitiva anterior debido a que el emplazamiento por edictos fue ineficaz, lo que obliga a reponer el procedimiento y establece los términos precisados en el considerando iv. esta decisión tiene importantes consecuencias para las partes involucradas.

### 2. datos del proceso
 tribunal: tribunal de justicia del estado de jalisco.
 expediente: no disponible en los hechos extraídos.
 etapa procesal: anulación de sentencia definitiva anterior.
 acción principal: reposición del procedimiento.

### 3. partes y documento base
 parte actora (acreedora): no disponible en los hechos extraídos.
 parte demandada (deudora): no disponible en los hechos extraídos.
 documento base: sentencia definitiva anterior.

### 4. hechos y decisión
hechos clave:
el emplazamiento por edictos no fue eficaz para encontrar al domicilio de la sociedad demandada, lo que llevó a búsquedas infructuosas en diferentes lugares. la parte actora presentó argumentos inoperantes e infundados o parcialmente fundados.

decisión final:
se deja insubsistente la sentencia definitiva anterior y se ordena reponer el procedimiento en los términos precisados en el considerando iv, sin hacer especial condena en contra de ninguna de las partes.

### 5. análisis y consecuencias
razonamiento del tribunal (análisis):
el tribunal anuló la sentencia definitiva anterior debido a que el emplazamiento por edictos no cumplió con los requisitos legales para notificar a la sociedad demandada, lo que invalida la decisión anterior. la falta de eficacia en el emplazamiento y las búsquedas infructuosas en diferentes lugares demuestran la ineficacia del procedimiento utilizado.

consecuencias financieras:
no se mencionan consecuencias financieras específicas, como suerte principal, intereses o costas. sin embargo, el reponer el procedimiento puede generar nuevos gastos y recursos para las partes involucradas.

### 6. conclusión
la sentencia del tribunal de justicia del estado de jalisco anula la sentencia definitiva anterior debido a la ineficacia del emplazamiento por edictos, lo que obliga a reponer el procedimiento y establece los términos precisados en el considerando iv. esta decisión tiene importantes consecuencias para las partes involucradas y puede generar nuevos gastos y recursos.

### 7. recomendaciones
se recomienda a las partes involucradas que revisen y ajusten sus procedimientos de notificación y emplazamiento para evitar situaciones similares en el futuro. además, se sugiere que las partes consideren la posibilidad de utilizar métodos alternativos de notificación más efectivos.

### 8. análisis adicional
el caso destaca la importancia de cumplir con los requisitos legales y procedimientos para garantizar la eficacia del emplazamiento y notificación en procesos judiciales. la ineficacia en este aspecto puede generar importantes consecuencias, incluyendo la anulación de sentencias definitivas y el reponer de procedimientos.

### 9. consideraciones finales
la decisión del tribunal tiene importantes implicaciones para las partes involucradas y demuestra la importancia de cumplir con los requisitos legales y procedimientos en procesos judiciales. se recomienda a las partes que revisen y ajusten sus prácticas para evitar situaciones similares en el futuro.

### 10. información adicional
no se proporciona información adicional relevante para este caso específico.


***
## 📚 Apéndice: Análisis Detallado de Precedentes Individuales

